# Script

## Bibliotecas

In [3108]:
import pandas as pd
import mysql.connector
import numpy as np
import clubes
import importlib
import os

Recarregando a informação que consta no arquivo "clubes.py".

In [3109]:
importlib.reload(clubes)
print(clubes.casa)

Benfica


## Partidas

### Configurando o banco de dados e carregando a consulta

In [3110]:
config = {
    "host": "127.0.0.1",
    "user": "admin",
    "password": "1234",
    "database": "bet_dados"
}

In [3111]:
conn = mysql.connector.connect(**config)

In [3112]:
query = f"""
SELECT
	data,
  temporada,
  id_jogo,
  campeonato,
  casa,
  gols_casa,
  gols_fora,
  fora,
  odd_casa,
  odd_empate,
  odd_fora,
  CASE
    WHEN gols_casa > gols_fora THEN casa
        WHEN gols_fora > gols_casa THEN fora
        WHEN gols_casa = gols_fora THEN 'Empate'
  END AS vencedor,
  gols_casa + gols_fora AS total_gols
FROM
	partidas
WHERE
	(casa = "{clubes.casa}" OR fora = "{clubes.casa}");
"""

In [3113]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_35332\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [3114]:
conn.close()

In [3115]:
df.head()

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols
0,2025-06-16,2025,13200247,Copa do Mundo de Clubes,Boca Juniors,2,2,Benfica,4.75,3.65,1.75,Empate,4
1,2025-06-20,2025,13200280,Copa do Mundo de Clubes,Benfica,6,0,Auckland City,1.01,29.00,67.00,Benfica,6
2,2025-06-24,2025,13200224,Copa do Mundo de Clubes,Benfica,1,0,Bayern de Munique,3.80,3.35,2.15,Benfica,1
3,2025-06-28,2025,13385899,Copa do Mundo de Clubes,Benfica,1,1,Chelsea,3.65,3.50,2.15,Empate,2
4,2024-08-11,2024/2025,12512900,Português,Famalicão,2,0,Benfica,NaN,NaN,NaN,Famalicão,2


### Tratando as informações

### Casa Atual

Criando uma cópia do DataFrame

In [3116]:
mask = (df["casa"] == clubes.casa) | (df["fora"] == clubes.casa)
casa = df.loc[mask].copy()

casa

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols
0,2025-06-16,2025,13200247,Copa do Mundo de Clubes,Boca Juniors,2,2,Benfica,4.75,3.65,1.75,Empate,4
1,2025-06-20,2025,13200280,Copa do Mundo de Clubes,Benfica,6,0,Auckland City,1.01,29.00,67.00,Benfica,6
2,2025-06-24,2025,13200224,Copa do Mundo de Clubes,Benfica,1,0,Bayern de Munique,3.80,3.35,2.15,Benfica,1
3,2025-06-28,2025,13385899,Copa do Mundo de Clubes,Benfica,1,1,Chelsea,3.65,3.50,2.15,Empate,2
4,2024-08-11,2024/2025,12512900,Português,Famalicão,2,0,Benfica,NaN,NaN,NaN,Famalicão,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,2025-09-20,2025,14131904,Português,AVS,0,3,Benfica,13.00,6.25,1.17,Benfica,3
290,2025-09-23,2025,14131859,Português,Benfica,1,1,Rio Ave,1.16,7.00,18.00,Empate,2
291,2025-09-26,2025,14131920,Português,Benfica,2,1,Gil Vicente,1.35,4.80,8.75,Benfica,3
292,2025-09-30,2025,14566581,Liga dos Campeões,Chelsea,1,0,Benfica,1.67,3.80,5.00,Chelsea,1


In [3117]:
casa.head()

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols
0,2025-06-16,2025,13200247,Copa do Mundo de Clubes,Boca Juniors,2,2,Benfica,4.75,3.65,1.75,Empate,4
1,2025-06-20,2025,13200280,Copa do Mundo de Clubes,Benfica,6,0,Auckland City,1.01,29.00,67.00,Benfica,6
2,2025-06-24,2025,13200224,Copa do Mundo de Clubes,Benfica,1,0,Bayern de Munique,3.80,3.35,2.15,Benfica,1
3,2025-06-28,2025,13385899,Copa do Mundo de Clubes,Benfica,1,1,Chelsea,3.65,3.50,2.15,Empate,2
4,2024-08-11,2024/2025,12512900,Português,Famalicão,2,0,Benfica,NaN,NaN,NaN,Famalicão,2


Selecionando a temporada atual

In [3118]:
casa = casa[casa["temporada"] == "2025"]

Contando os jogos

In [3119]:
jogos_casa = casa.shape[0]

jogos_casa

18

Contando as vitórias

In [3120]:
vitorias = casa[casa["vencedor"] == clubes.casa]

vitorias_casa = vitorias.shape[0]

vitorias_casa

10

Contando os empates

In [3121]:
empates = casa[casa["vencedor"] == "Empate"]

empates_casa = empates.shape[0]

empates_casa

6

Contando as derrotas

In [3122]:
derrotas = casa[(casa["vencedor"] != clubes.casa) & (casa["vencedor"] != "Empate")]

derrotas_casa = derrotas.shape[0]

derrotas_casa

2

Gols marcados

In [3123]:
gols_casa = casa.loc[casa["casa"] == clubes.casa, "gols_casa"].sum()
gols_fora = casa.loc[casa["fora"] == clubes.casa, "gols_fora"].sum()

gols_marcados_casa = gols_casa + gols_fora

gols_marcados_casa

30

Gols sofridos

In [3124]:
gols_sofridos_casa = casa.loc[casa["casa"] == clubes.casa, "gols_fora"].sum()
gols_sofridos_fora = casa.loc[casa["fora"] == clubes.casa, "gols_casa"].sum()

gols_sofridos_casa = gols_sofridos_casa + gols_sofridos_fora

gols_sofridos_casa

11

Aproveitamento

In [3125]:
if jogos_casa == 0:
    aproveitamento_casa = 0
else:
    aproveitamento_casa = round((vitorias_casa * 3 + empates_casa) / (jogos_casa * 3) * 100, 2)

aproveitamento_casa

66.67

Over 2.5 gols

In [3126]:
over_2_casa = casa[casa["total_gols"] >= 3].shape[0]

over_2_casa

7

% de jogos com 3 ou mais gols

In [3127]:
if jogos_casa == 0:
    prob_over_2_casa = 0
else:
    prob_over_2_casa = round((over_2_casa / jogos_casa) * 100, 2)

prob_over_2_casa

38.89

Under 2.5 gols

In [3128]:
under_2_casa = casa[casa["total_gols"] < 3].shape[0]

under_2_casa

11

% de jogos com menos de 3 gols

In [3129]:
if jogos_casa == 0:
    prob_under_2_casa = 0
else:
    prob_under_2_casa = round((under_2_casa / jogos_casa) * 100, 2)

prob_under_2_casa

61.11

Odd média

In [3130]:
odd_media_casa = casa[casa["casa"] == clubes.casa]["odd_casa"].mean()
odd_media_fora = casa[casa["fora"] == clubes.casa]["odd_fora"].mean()

odd_media_casa = round((odd_media_casa + odd_media_fora) / 2, 2)

odd_media_casa

2.07

Odd média quando venceu

In [3131]:
odd_media_vitoria_casa = vitorias[casa["casa"] == clubes.casa]["odd_casa"].mean()
odd_media_vitoria_fora = vitorias[casa["fora"] == clubes.casa]["odd_fora"].mean()

odd_media_vitoria_casa = round(odd_media_vitoria_casa + odd_media_vitoria_fora / 2, 2)

odd_media_vitoria_casa

C:\Users\stefa\AppData\Local\Temp\ipykernel_35332\524038383.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  odd_media_vitoria_casa = vitorias[casa["casa"] == clubes.casa]["odd_casa"].mean()
C:\Users\stefa\AppData\Local\Temp\ipykernel_35332\524038383.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  odd_media_vitoria_fora = vitorias[casa["fora"] == clubes.casa]["odd_fora"].mean()


2.49

Contando as goleadas

In [3132]:
goleadas_mandante_casa = casa[(casa["casa"] == clubes.casa) & (casa["gols_casa"] >= 4)].shape[0]
goleadas_mandante_fora = casa[(casa["fora"] == clubes.casa) & (casa["gols_fora"] >= 4)].shape[0]

goleadas_casa = goleadas_mandante_casa + goleadas_mandante_fora

goleadas_casa

1

% de goleadas aplicadas

In [3133]:
if jogos_casa == 0:
    prob_goleada_casa = 0
else:
    prob_goleada_casa = round((goleadas_casa / jogos_casa) * 100, 2)

prob_goleada_casa

5.56

Montando o DataFrame do time da casa

In [3134]:
dados = {
    "Jogos": jogos_casa,
    "Vitórias": vitorias_casa,
    "Empates": empates_casa,
    "Derrotas": derrotas_casa,
    "Gols marcados": gols_marcados_casa,
    "Gols sofridos": gols_sofridos_casa,
    "Aproveitamento": aproveitamento_casa,
    "Over 2.5 gols": over_2_casa,
    "Probabilidade de over 2.5 gols": prob_over_2_casa,
    "Under 2.5 gols": under_2_casa,
    "Probabilidade de under 2.5 gols": prob_under_2_casa,
    "Odd média": odd_media_casa,
    "Odd média quando venceu": odd_media_vitoria_casa,
    "Goleadas": goleadas_casa,
    "Probabilidade de golear": prob_goleada_casa,
    "Goleadas como mandante": goleadas_mandante_casa,
    "Goleadas como visitante": goleadas_mandante_fora
}

In [3135]:
ordem = [
    "Jogos", "Vitórias", "Empates", "Derrotas",
    "Gols marcados", "Gols sofridos",
    "Aproveitamento",
    "Over 2.5 gols", "Probabilidade de over 2.5 gols",
    "Under 2.5 gols", "Probabilidade de under 2.5 gols",
    "Odd média", "Odd média quando venceu",
    "Goleadas", "Probabilidade de golear",
]

faltando = [k for k in ordem if k not in dados]
if faltando:
    raise KeyError(f"As seguintes chaves não estão no dicionário 'dados': {faltando}")


dados_ordenados = {k: dados[k] for k in ordem}
df_final_partidas = pd.DataFrame.from_dict(
    dados_ordenados, orient="index", columns=[clubes.casa]
)


variaveis_float = {
    "aproveitamento",
    "probabilidade de over 2.5 gols",
    "probabilidade de under 2.5 gols",
    "odd média",
    "odd média quando venceu",
    "probabilidade de golear",
}

def _fmt(nome, valor):
    if str(nome).lower() in variaveis_float:
        return f"{float(valor):.2f}"
    else:
        return f"{int(valor)}"

df_final_partidas[clubes.casa] = [
    _fmt(nome, valor) for nome, valor in zip(df_final_partidas.index, df_final_partidas[clubes.casa])
]

# Índice vira coluna "Informações"
tabela_casa = df_final_partidas.reset_index().rename(columns={"index": "Informações"})

In [3136]:
tabela_casa.style.hide(axis="index")

Informações,Benfica
Jogos,18
Vitórias,10
Empates,6
Derrotas,2
Gols marcados,30
Gols sofridos,11
Aproveitamento,66.67
Over 2.5 gols,7
Probabilidade de over 2.5 gols,38.89
Under 2.5 gols,11


In [3137]:
caminho = r"C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx"

In [3138]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    tabela_casa.to_excel(writer, sheet_name="Partidas Atual", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Casa (últimos 10 jogos)

In [3139]:
casa = casa.sort_values("data", ascending=False).head(10)

casa

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols
293,2025-10-05,2025,14131931,Português,Porto,0,0,Benfica,1.91,3.50,3.75,Empate,0
292,2025-09-30,2025,14566581,Liga dos Campeões,Chelsea,1,0,Benfica,1.67,3.80,5.00,Chelsea,1
291,2025-09-26,2025,14131920,Português,Benfica,2,1,Gil Vicente,1.35,4.80,8.75,Benfica,3
290,2025-09-23,2025,14131859,Português,Benfica,1,1,Rio Ave,1.16,7.00,18.00,Empate,2
289,2025-09-20,2025,14131904,Português,AVS,0,3,Benfica,13.00,6.25,1.17,Benfica,3
288,2025-09-16,2025,14566738,Liga dos Campeões,Benfica,2,3,Qarabag,1.17,8.00,15.00,Qarabag,5
287,2025-09-12,2025,14131901,Português,Benfica,1,1,Santa Clara,1.33,4.60,11.00,Empate,2
286,2025-08-31,2025,14131888,Português,Alverca,1,2,Benfica,13.00,5.90,1.23,Benfica,3
285,2025-08-27,2025,14348673,Liga dos Campeões,Benfica,1,0,Fenerbahçe,1.75,3.55,4.90,Benfica,1
284,2025-08-23,2025,14131880,Português,Benfica,3,0,Tondela,1.17,7.00,15.00,Benfica,3


Contando os jogos

In [3140]:
jogos_casa = casa.shape[0]

jogos_casa

10

Contando as vitórias

In [3141]:
vitorias = casa[casa["vencedor"] == clubes.casa]

vitorias_casa = vitorias.shape[0]

vitorias_casa

5

Contando os empates

In [3142]:
empates = casa[casa["vencedor"] == "Empate"]

empates_casa = empates.shape[0]

empates_casa

3

Contando as derrotas

In [3143]:
derrotas = casa[(casa["vencedor"] != clubes.casa) & (casa["vencedor"] != "Empate")]

derrotas_casa = derrotas.shape[0]

derrotas_casa

2

Gols marcados

In [3144]:
gols_casa = casa.loc[casa["casa"] == clubes.casa, "gols_casa"].sum()
gols_fora = casa.loc[casa["fora"] == clubes.casa, "gols_fora"].sum()

gols_marcados_casa = gols_casa + gols_fora

gols_marcados_casa

15

Gols sofridos

In [3145]:
gols_sofridos_casa = casa.loc[casa["casa"] == clubes.casa, "gols_fora"].sum()
gols_sofridos_fora = casa.loc[casa["fora"] == clubes.casa, "gols_casa"].sum()

gols_sofridos_casa = gols_sofridos_casa + gols_sofridos_fora

gols_sofridos_casa

8

Aproveitamento

In [3146]:
aproveitamento_casa = round((vitorias_casa * 3 + empates_casa) / (jogos_casa * 3) * 100, 2)

aproveitamento_casa

60.0

Over 2.5 gols

In [3147]:
over_2_casa = casa[casa["total_gols"] >= 3].shape[0]

over_2_casa

5

% de jogos com 3 ou mais gols

In [3148]:
prob_over_2_casa = round((over_2_casa / jogos_casa) * 100, 2)

prob_over_2_casa

50.0

Under 2.5 gols

In [3149]:
under_2_casa = casa[casa["total_gols"] < 3].shape[0]

under_2_casa

5

% de jogos com menos de 3 gols

In [3150]:
prob_under_2_casa = round((under_2_casa / jogos_casa) * 100, 2)

prob_under_2_casa

50.0

Odd média

In [3151]:
odd_media_casa = casa[casa["casa"] == clubes.casa]["odd_casa"].mean()
odd_media_fora = casa[casa["fora"] == clubes.casa]["odd_fora"].mean()

odd_media_casa = round((odd_media_casa + odd_media_fora) / 2, 2)

odd_media_casa

2.05

Odd média quando venceu

In [3152]:
odd_media_vitoria_casa = vitorias[casa["casa"] == clubes.casa]["odd_casa"].mean()
odd_media_vitoria_fora = vitorias[casa["fora"] == clubes.casa]["odd_fora"].mean()

odd_media_vitoria_casa = round(odd_media_vitoria_casa + odd_media_vitoria_fora / 2, 2)

odd_media_vitoria_casa

C:\Users\stefa\AppData\Local\Temp\ipykernel_35332\524038383.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  odd_media_vitoria_casa = vitorias[casa["casa"] == clubes.casa]["odd_casa"].mean()
C:\Users\stefa\AppData\Local\Temp\ipykernel_35332\524038383.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  odd_media_vitoria_fora = vitorias[casa["fora"] == clubes.casa]["odd_fora"].mean()


2.02

Contando as goleadas

In [3153]:
goleadas_mandante_casa = casa[(casa["casa"] == clubes.casa) & (casa["gols_casa"] >= 4)].shape[0]
goleadas_mandante_fora = casa[(casa["fora"] == clubes.casa) & (casa["gols_fora"] >= 4)].shape[0]

goleadas_casa = goleadas_mandante_casa + goleadas_mandante_fora

goleadas_casa

0

% de goleadas aplicadas

In [3154]:
prob_goleada_casa = round((goleadas_casa / jogos_casa) * 100, 2)

prob_goleada_casa

0.0

Montando o DataFrame do time da casa

In [3155]:
dados = {
    "Jogos": jogos_casa,
    "Vitórias": vitorias_casa,
    "Empates": empates_casa,
    "Derrotas": derrotas_casa,
    "Gols marcados": gols_marcados_casa,
    "Gols sofridos": gols_sofridos_casa,
    "Aproveitamento": aproveitamento_casa,
    "Over 2.5 gols": over_2_casa,
    "Probabilidade de over 2.5 gols": prob_over_2_casa,
    "Under 2.5 gols": under_2_casa,
    "Probabilidade de under 2.5 gols": prob_under_2_casa,
    "Odd média": odd_media_casa,
    "Odd média quando venceu": odd_media_vitoria_casa,
    "Goleadas": goleadas_casa,
    "Probabilidade de golear": prob_goleada_casa,
    "Goleadas como mandante": goleadas_mandante_casa,
    "Goleadas como visitante": goleadas_mandante_fora
}

In [3156]:
ordem = [
    "Jogos", "Vitórias", "Empates", "Derrotas",
    "Gols marcados", "Gols sofridos",
    "Aproveitamento",
    "Over 2.5 gols", "Probabilidade de over 2.5 gols",
    "Under 2.5 gols", "Probabilidade de under 2.5 gols",
    "Odd média", "Odd média quando venceu",
    "Goleadas", "Probabilidade de golear",
]

faltando = [k for k in ordem if k not in dados]
if faltando:
    raise KeyError(f"As seguintes chaves não estão no dicionário 'dados': {faltando}")


dados_ordenados = {k: dados[k] for k in ordem}
df_final_partidas = pd.DataFrame.from_dict(
    dados_ordenados, orient="index", columns=[clubes.casa]
)


variaveis_float = {
    "aproveitamento",
    "probabilidade de over 2.5 gols",
    "probabilidade de under 2.5 gols",
    "odd média",
    "odd média quando venceu",
    "probabilidade de golear",
}

def _fmt(nome, valor):
    if str(nome).lower() in variaveis_float:
        return f"{float(valor):.2f}"
    else:
        return f"{int(valor)}"

df_final_partidas[clubes.casa] = [
    _fmt(nome, valor) for nome, valor in zip(df_final_partidas.index, df_final_partidas[clubes.casa])
]

# Índice vira coluna "Informações"
tabela_recente = df_final_partidas.reset_index().rename(columns={"index": "Informações"})

In [3157]:
tabela_recente.style.hide(axis="index")

Informações,Benfica
Jogos,10
Vitórias,5
Empates,3
Derrotas,2
Gols marcados,15
Gols sofridos,8
Aproveitamento,60.00
Over 2.5 gols,5
Probabilidade de over 2.5 gols,50.00
Under 2.5 gols,5


Salvando o arquivo

In [3158]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    tabela_recente.to_excel(writer, sheet_name="Desempenho 10 Jogos", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Clean sheets e clean goals (10 jogos)

In [3159]:
def clean_stats_do_clube(df, clube) -> tuple[int, int]:
    # Retorna (clean_sheets, clean_goals) para 'clube'
    if clube is None or df is None or df.empty:
        return 0, 0

    m = (df["casa"] == clube) | (df["fora"] == clube)
    jogos = df.loc[m].copy()
    if jogos.empty:
        return 0, 0

    gols_favor = np.where(jogos["casa"] == clube, jogos["gols_casa"], jogos["gols_fora"])
    gols_contra = np.where(jogos["casa"] == clube, jogos["gols_fora"], jogos["gols_casa"])

    gols_favor = pd.Series(gols_favor, dtype="float").fillna(0)
    gols_contra = pd.Series(gols_contra, dtype="float").fillna(0)

    clean_sheets = int((gols_contra == 0).sum())  # não sofreu
    clean_goals  = int((gols_favor  == 0).sum())  # não marcou
    return clean_sheets, clean_goals


def montar_df_clean_unificado(df_casa, clubes, df_fora=None):
    """
    Retorna somente as estatísticas do clube mandante.
    Aceita chamada com 2 args (df_casa, clubes) ou 3 args (df_casa, df_fora, clubes).
    O df_fora é ignorado de propósito.
    """
    # Se o usuário chamar com a assinatura antiga (df_casa, df_fora, clubes), reordena:
    if df_fora is not None and not isinstance(clubes, (dict, str, type(None))):
        # Chamaram como (df_casa, df_fora, clubes) -> clubes está no 3º arg
        df_casa, df_fora, clubes = df_casa, clubes, df_fora  # swap para compatibilidade

    clube_casa = getattr(clubes, "casa", None) if not isinstance(clubes, dict) else clubes.get("casa")

    colunas_necessarias = {"casa", "fora", "gols_casa", "gols_fora"}
    faltantes = colunas_necessarias - set(df_casa.columns)
    if faltantes:
        raise KeyError(f"df_casa está sem as colunas: {sorted(faltantes)}")

    cs_casa, cg_casa = clean_stats_do_clube(df_casa, clube_casa)

    resultado = pd.DataFrame(
        [{"clube": clube_casa, "clean_sheets": cs_casa, "clean_goals": cg_casa}],
        columns=["clube", "clean_sheets", "clean_goals"],
    )
    return resultado


In [3160]:
df_cleans = montar_df_clean_unificado(casa, clubes)

df_cleans

,clube,clean_sheets,clean_goals
0,Benfica,4,2


### Força no mando

#### Carregando a consulta novamente

In [3161]:
conn = mysql.connector.connect(**config)

In [3162]:
query = f"""
SELECT
	data,
  temporada,
  id_jogo,
  campeonato,
  casa,
  gols_casa,
  gols_fora,
  fora,
  odd_casa,
  odd_empate,
  odd_fora,
  CASE
    WHEN gols_casa > gols_fora THEN casa
        WHEN gols_fora > gols_casa THEN fora
        WHEN gols_casa = gols_fora THEN 'Empate'
  END AS vencedor,
  gols_casa + gols_fora AS total_gols
FROM
	partidas
WHERE
	(casa = "{clubes.casa}" OR fora = "{clubes.casa}");
"""

In [3163]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_35332\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [3164]:
conn.close()

In [3165]:
df.head()

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols
0,2025-06-16,2025,13200247,Copa do Mundo de Clubes,Boca Juniors,2,2,Benfica,4.75,3.65,1.75,Empate,4
1,2025-06-20,2025,13200280,Copa do Mundo de Clubes,Benfica,6,0,Auckland City,1.01,29.00,67.00,Benfica,6
2,2025-06-24,2025,13200224,Copa do Mundo de Clubes,Benfica,1,0,Bayern de Munique,3.80,3.35,2.15,Benfica,1
3,2025-06-28,2025,13385899,Copa do Mundo de Clubes,Benfica,1,1,Chelsea,3.65,3.50,2.15,Empate,2
4,2024-08-11,2024/2025,12512900,Português,Famalicão,2,0,Benfica,NaN,NaN,NaN,Famalicão,2


#### Carregando o time da casa

In [3166]:
casa = df.copy()

Selecionando o time da casa e filtrando os últimos 10 jogos

In [3167]:
casa = casa[casa["casa"] == clubes.casa]

In [3168]:
casa = casa.sort_values("data", ascending=False).head(10)

In [3169]:
def func_desempenho_casa(casa_df):
    gc = casa_df["gols_casa"].apply(pd.to_numeric, errors="coerce").fillna(0)
    gf = casa_df["gols_fora"].apply(pd.to_numeric, errors="coerce").fillna(0)

    jogos     = int(len(casa_df))
    vitorias  = int((gc > gf).sum())
    empates   = int((gc == gf).sum())
    derrotas  = int((gc < gf).sum())
    marcados  = int(gc.sum())
    sofridos  = int(gf.sum())

    # Odds
    odd_casa = casa_df["odd_casa"].apply(pd.to_numeric, errors="coerce")
    odd_media = odd_casa.mean(skipna=True)
    odd_media_vitoria = odd_casa[gc > gf].mean(skipna=True)
    # evita NaN quando não houver vitórias
    odd_media_vitoria = 0.0 if pd.isna(odd_media_vitoria) else odd_media_vitoria

    denom = 30 if jogos >= 10 else max(jogos * 3, 1)
    aproveitamento = ((vitorias * 3 + empates) / denom) * 100          # 0–100
    media_marcados = (marcados / jogos) if jogos else 0.0
    media_sofridos = (sofridos / jogos) if jogos else 0.0

    clean_sheets = int((gf == 0).sum())
    clean_goals  = int((gc == 0).sum())

    linhas = [
        {"informacao": "Jogos",                        "dados": jogos},
        {"informacao": "Vitórias",                     "dados": vitorias},
        {"informacao": "Empates",                      "dados": empates},
        {"informacao": "Derrotas",                     "dados": derrotas},
        {"informacao": "Gols marcados",                "dados": marcados},
        {"informacao": "Gols sofridos",                "dados": sofridos},
        {"informacao": "Aproveitamento",               "dados": round(aproveitamento, 2)},
        {"informacao": "Média de gols marcados",       "dados": round(media_marcados, 2)},
        {"informacao": "Média de gols sofridos",       "dados": round(media_sofridos, 2)},
        {"informacao": "Clean sheets",                 "dados": clean_sheets},
        {"informacao": "Clean goals",                  "dados": clean_goals},
        {"informacao": "Odd média",                    "dados": round(float(odd_media) if not pd.isna(odd_media) else 0.0, 2)},
        {"informacao": "Odd média quando venceu",      "dados": round(float(odd_media_vitoria), 2)},
    ]

    df = pd.DataFrame(linhas, columns=["informacao", "dados"])

    # Inteiros obrigatórios
    int_infos = {
        "Jogos","Vitórias","Empates","Derrotas",
        "Gols marcados","Gols sofridos","Clean sheets","Clean goals"
    }
    mask_int = df["informacao"].isin(int_infos)
    df.loc[mask_int, "dados"] = df.loc[mask_int, "dados"].astype(int)

    # Formatação para exibição (2 casas só nos floats)
    float_infos = {
        "Aproveitamento", "Média de gols marcados", "Média de gols sofridos",
        "Odd média", "Odd média quando venceu"
    }
    df["dados_fmt"] = df.apply(
        lambda r: f"{r['dados']:.2f}" if r["informacao"] in float_infos else f"{int(r['dados'])}",
        axis=1
    )

    return df

In [3170]:
desempenho_casa = func_desempenho_casa(casa)

desempenho_casa = desempenho_casa[["informacao", "dados_fmt"]]

desempenho_casa = desempenho_casa.rename(columns={"dados_fmt": "dados"})

desempenho_casa

,informacao,dados
0,Jogos,10
1,Vitórias,6
2,Empates,3
3,Derrotas,1
4,Gols marcados,20
5,Gols sofridos,7
6,Aproveitamento,70.00
7,Média de gols marcados,2.00
8,Média de gols sofridos,0.70
9,Clean sheets,5


### Confronto direto

#### Carregando a consulta

In [3171]:
conn = mysql.connector.connect(**config)

In [3172]:
query = f"""
SELECT
	data,
  temporada,
  id_jogo,
  campeonato,
  casa,
  gols_casa,
  gols_fora,
  fora,
  odd_casa,
  odd_empate,
  odd_fora,
  CASE
    WHEN gols_casa > gols_fora THEN casa
        WHEN gols_fora > gols_casa THEN fora
        WHEN gols_casa = gols_fora THEN 'Empate'
  END AS vencedor,
  gols_casa + gols_fora AS total_gols
FROM
	partidas
WHERE
	(casa = "{clubes.casa}" OR fora = "{clubes.casa}")
    AND (casa = "{clubes.fora}" OR fora = "{clubes.fora}");
"""

In [3173]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_35332\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [3174]:
df.head()

,data,temporada,id_jogo,campeonato,casa,gols_casa,gols_fora,fora,odd_casa,odd_empate,odd_fora,vencedor,total_gols
0,2023-11-04,2023/2024,11409142,Português,Chaves,0,2,Benfica,NaN,NaN,NaN,Benfica,2
1,2024-03-29,2023/2024,11408419,Português,Benfica,1,0,Chaves,1.14,9.00,17.00,Benfica,1
2,2022-10-29,2022/2023,10437928,Português,Benfica,5,0,Chaves,1.10,9.00,19.00,Benfica,5
3,2023-04-15,2022/2023,10438105,Português,Chaves,1,0,Benfica,10.00,5.75,1.25,Chaves,1


#### H2H - Geral

In [3175]:
confronto_direto = df.copy()

In [3176]:
def func_desempenho_confronto_direto(confronto_direto, clubes):
    # Gols como numérico
    gc = confronto_direto["gols_casa"].apply(pd.to_numeric, errors="coerce").fillna(0)
    gf = confronto_direto["gols_fora"].apply(pd.to_numeric, errors="coerce").fillna(0)

    # Máscaras de quem é quem em cada linha
    m_casa_home = confronto_direto["casa"] == clubes.casa
    m_casa_away = confronto_direto["fora"] == clubes.casa
    m_fora_home = confronto_direto["casa"] == clubes.fora
    m_fora_away = confronto_direto["fora"] == clubes.fora

    jogos = int(len(confronto_direto))

    # Vitórias corretas (independe de onde jogou)
    vitorias_casa = int(((m_casa_home & (gc > gf)) | (m_casa_away & (gf > gc))).sum())
    empates       = int((gc == gf).sum())
    vitorias_fora = int(((m_fora_home & (gc > gf)) | (m_fora_away & (gf > gc))).sum())

    # Gols por clube (soma do lado correspondente em cada linha)
    gols_do_casa = int(gc[m_casa_home].sum() + gf[m_casa_away].sum())
    gols_do_fora = int(gc[m_fora_home].sum() + gf[m_fora_away].sum())

    # Média de gols do duelo
    if "total_gols" in confronto_direto.columns:
        total_gols = confronto_direto["total_gols"].apply(pd.to_numeric, errors="coerce")
    else:
        total_gols = gc + gf
    media_gols_duelo = round(total_gols.mean(skipna=True) if jogos else 0.0, 2)

    # ===== Odds (condicionadas ao lado) =====
    odd_casa_col = confronto_direto["odd_casa"].apply(pd.to_numeric, errors="coerce")
    odd_fora_col = confronto_direto["odd_fora"].apply(pd.to_numeric, errors="coerce")

    # Odd média do clubes.casa: quando ele é mandante usa odd_casa; quando visitante usa odd_fora
    odds_clubes_casa = pd.concat([
        odd_casa_col[m_casa_home],
        odd_fora_col[m_casa_away]
    ])
    odd_media_clubes_casa = float(odds_clubes_casa.mean(skipna=True) or 0.0)

    # Odd média do clubes.fora: análogo
    odds_clubes_fora = pd.concat([
        odd_casa_col[m_fora_home],
        odd_fora_col[m_fora_away]
    ])
    odd_media_clubes_fora = float(odds_clubes_fora.mean(skipna=True) or 0.0)

    # Odd média do vencedor (ignora empates)
    home_win = gc > gf
    away_win = gf > gc
    odds_vencedor = pd.concat([
        odd_casa_col[home_win],
        odd_fora_col[away_win]
    ])
    odd_media_vencedor = float(odds_vencedor.mean(skipna=True) or 0.0)

    # ===== Monta saída =====
    linhas = [
        {"informacao": "Jogos",                                 "dados": jogos},
        {"informacao": f"Vitórias do {clubes.casa}",            "dados": vitorias_casa},
        {"informacao": "Empates",                               "dados": empates},
        {"informacao": f"Vitórias do {clubes.fora}",            "dados": vitorias_fora},
        {"informacao": f"Gols do {clubes.casa}",                "dados": gols_do_casa},
        {"informacao": f"Gols do {clubes.fora}",                "dados": gols_do_fora},
        {"informacao": "Média de gols do duelo",                "dados": media_gols_duelo},
        {"informacao": f"Odd média do {clubes.casa}",           "dados": round(odd_media_clubes_casa, 2)},
        {"informacao": f"Odd média do {clubes.fora}",           "dados": round(odd_media_clubes_fora, 2)},
        {"informacao": "Odd média do vencedor",                 "dados": round(odd_media_vencedor, 2)},
    ]
    df = pd.DataFrame(linhas, columns=["informacao", "dados"])

    # Inteiros garantidos
    int_infos = {
        "Jogos",
        f"Vitórias do {clubes.casa}", "Empates", f"Vitórias do {clubes.fora}",
        f"Gols do {clubes.casa}", f"Gols do {clubes.fora}"
    }
    df.loc[df["informacao"].isin(int_infos), "dados"] = df.loc[df["informacao"].isin(int_infos), "dados"].astype(int)

    # Formatação de exibição: 2 casas para médias/odds
    float_infos = {
        "Média de gols do duelo",
        f"Odd média do {clubes.casa}",
        f"Odd média do {clubes.fora}",
        "Odd média do vencedor"
    }
    df["dados_fmt"] = df.apply(
        lambda r: f"{r['dados']:.2f}" if r["informacao"] in float_infos else f"{int(r['dados'])}",
        axis=1
    )

    return df

In [3177]:
desempenho_confronto = func_desempenho_confronto_direto(confronto_direto, clubes)

desempenho_confronto = desempenho_confronto[["informacao", "dados_fmt"]]

desempenho_confronto = desempenho_confronto.rename(columns={"dados_fmt": "dados"})

desempenho_confronto

,informacao,dados
0,Jogos,4
1,Vitórias do Benfica,3
2,Empates,0
3,Vitórias do Chaves,1
4,Gols do Benfica,8
5,Gols do Chaves,1
6,Média de gols do duelo,2.25
7,Odd média do Benfica,1.16
8,Odd média do Chaves,15.33
9,Odd média do vencedor,4.08


#### H2H - últimos 10 jogos

In [3178]:
confronto_recente = df.copy()

In [3179]:
confronto_recente = confronto_recente.sort_values("data", ascending=False).head(10)

In [3180]:
def func_desempenho_confronto_recente(confronto_recente, clubes):
    # ---- Converte gols para numérico ----
    gc = confronto_recente["gols_casa"].apply(pd.to_numeric, errors="coerce").fillna(0)
    gf = confronto_recente["gols_fora"].apply(pd.to_numeric, errors="coerce").fillna(0)

    # ---- Máscaras: posição de cada clube em cada jogo ----
    m_casa_home = confronto_recente["casa"] == clubes.casa
    m_casa_away = confronto_recente["fora"] == clubes.casa
    m_fora_home = confronto_recente["casa"] == clubes.fora
    m_fora_away = confronto_recente["fora"] == clubes.fora

    jogos = int(len(confronto_recente))

    # ---- Vitórias/Empates (independe de onde jogou) ----
    vitorias_casa = int(((m_casa_home & (gc > gf)) | (m_casa_away & (gf > gc))).sum())
    empates       = int((gc == gf).sum())
    vitorias_fora = int(((m_fora_home & (gc > gf)) | (m_fora_away & (gf > gc))).sum())

    # ---- Gols por clube (lado correto em cada linha) ----
    gols_do_casa = int(gc[m_casa_home].sum() + gf[m_casa_away].sum())
    gols_do_fora = int(gc[m_fora_home].sum() + gf[m_fora_away].sum())

    # ---- Média de gols do duelo ----
    if "total_gols" in confronto_recente.columns:
        total_gols = confronto_recente["total_gols"].apply(pd.to_numeric, errors="coerce")
    else:
        total_gols = gc + gf
    media_gols_duelo = round(total_gols.mean(skipna=True) if jogos else 0.0, 2)

    # ---- Odds (usando a odd do lado correto em cada jogo) ----
    odd_casa_col = confronto_recente["odd_casa"].apply(pd.to_numeric, errors="coerce")
    odd_fora_col = confronto_recente["odd_fora"].apply(pd.to_numeric, errors="coerce")

    # Odd média do clubes.casa
    odds_clubes_casa = pd.concat([odd_casa_col[m_casa_home], odd_fora_col[m_casa_away]])
    odd_media_clubes_casa = float(odds_clubes_casa.mean(skipna=True) or 0.0)

    # Odd média do clubes.fora
    odds_clubes_fora = pd.concat([odd_casa_col[m_fora_home], odd_fora_col[m_fora_away]])
    odd_media_clubes_fora = float(odds_clubes_fora.mean(skipna=True) or 0.0)

    # Odd média do vencedor (ignora empates)
    home_win = gc > gf
    away_win = gf > gc
    odds_vencedor = pd.concat([odd_casa_col[home_win], odd_fora_col[away_win]])
    odd_media_vencedor = float(odds_vencedor.mean(skipna=True) or 0.0)

    # ---- Monta saída ----
    linhas = [
        {"informacao": "Jogos",                                 "dados": jogos},
        {"informacao": f"Vitórias do {clubes.casa}",            "dados": vitorias_casa},
        {"informacao": "Empates",                               "dados": empates},
        {"informacao": f"Vitórias do {clubes.fora}",            "dados": vitorias_fora},
        {"informacao": f"Gols do {clubes.casa}",                "dados": gols_do_casa},
        {"informacao": f"Gols do {clubes.fora}",                "dados": gols_do_fora},
        {"informacao": "Média de gols do duelo",                "dados": media_gols_duelo},
        {"informacao": f"Odd média do {clubes.casa}",           "dados": round(odd_media_clubes_casa, 2)},
        {"informacao": f"Odd média do {clubes.fora}",           "dados": round(odd_media_clubes_fora, 2)},
        {"informacao": "Odd média do vencedor",                 "dados": round(odd_media_vencedor, 2)},
    ]
    df = pd.DataFrame(linhas, columns=["informacao", "dados"])

    # Inteiros garantidos
    int_infos = {
        "Jogos",
        f"Vitórias do {clubes.casa}", "Empates", f"Vitórias do {clubes.fora}",
        f"Gols do {clubes.casa}", f"Gols do {clubes.fora}"
    }
    df.loc[df["informacao"].isin(int_infos), "dados"] = df.loc[df["informacao"].isin(int_infos), "dados"].astype(int)

    # Formatação de exibição (2 casas para médias/odds)
    float_infos = {
        "Média de gols do duelo",
        f"Odd média do {clubes.casa}",
        f"Odd média do {clubes.fora}",
        "Odd média do vencedor"
    }
    df["dados_fmt"] = df.apply(
        lambda r: f"{r['dados']:.2f}" if r["informacao"] in float_infos else f"{int(r['dados'])}",
        axis=1
    )
    return df

In [3181]:
historico_recente = func_desempenho_confronto_recente(confronto_recente, clubes)

historico_recente = historico_recente[["informacao", "dados_fmt"]]

historico_recente = historico_recente.rename(columns={"dados_fmt": "dados"})

historico_recente

,informacao,dados
0,Jogos,4
1,Vitórias do Benfica,3
2,Empates,0
3,Vitórias do Chaves,1
4,Gols do Benfica,8
5,Gols do Chaves,1
6,Média de gols do duelo,2.25
7,Odd média do Benfica,1.16
8,Odd média do Chaves,15.33
9,Odd média do vencedor,4.08


## 1º Tempo

### Carregando a consulta

In [3182]:
conn = mysql.connector.connect(**config)

In [3183]:
query = f"""
SELECT
    p.data,
    t1.temporada,
    t1.clube,
    CASE
        WHEN t1.clube = p.casa THEN 'Casa'
        WHEN t1.clube = p.fora THEN 'Fora'
        ELSE 'Verificar'
    END AS mando,
    t1.ball_possession,
    t1.expected_goals,
    t1.big_chances,
    t1.total_shots,
    t1.shots_on_target,
    t1.shots_inside_box,
    t1.hit_woodwork,
    t1.corner_kicks,
    t1.passes,
    t1.big_chances_scored,
    t1.accurate_passes,
    t1.through_balls,
    t1.final_third_entries,
    t1.fouled_in_final_third,
    t1.dispossessed,
    t1.duels,
    t1.yellow_cards,
    t1.red_cards
FROM
    1_tempo t1
    INNER JOIN partidas p ON t1.id_jogo = p.id_jogo
WHERE
    t1.clube IN ("{clubes.casa}");
"""

In [3184]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_35332\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [3185]:
conn.close()

In [3186]:
df = df[df["temporada"] == "2025"]

In [3187]:
df.head()

,data,temporada,clube,mando,ball_possession,expected_goals,big_chances,total_shots,shots_on_target,shots_inside_box,hit_woodwork,corner_kicks,passes,big_chances_scored,accurate_passes,through_balls,final_third_entries,fouled_in_final_third,dispossessed,duels,yellow_cards,red_cards
0,2025-06-16,2025,Benfica,Fora,59%,0.0,1,3,1,3,0,3,229,1,193,0,33,2,7,51%,0,0
1,2025-06-20,2025,Benfica,Casa,64%,0.0,5,18,8,10,1,5,250,1,222,1,32,2,2,49%,1,0
2,2025-06-24,2025,Benfica,Casa,30%,0.0,0,4,3,1,0,3,149,0,124,0,16,1,5,63%,1,0
3,2025-06-28,2025,Benfica,Casa,39%,0.0,0,2,1,1,0,0,203,0,171,0,15,1,4,56%,0,0
109,2025-07-31,2025,Benfica,Fora,49%,0.0,0,4,0,1,0,3,203,0,172,0,27,3,4,52%,1,0


### Casa

Jogos

In [3188]:
# Filtrando apenas informações do time mandante
casa = df[df["clube"] == clubes.casa]

# Selecionando apenas os últimos 10 jogos
casa = casa.sort_values("data", ascending=False).head(10)

casa.head(3)

,data,temporada,clube,mando,ball_possession,expected_goals,big_chances,total_shots,shots_on_target,shots_inside_box,hit_woodwork,corner_kicks,passes,big_chances_scored,accurate_passes,through_balls,final_third_entries,fouled_in_final_third,dispossessed,duels,yellow_cards,red_cards
288,2025-10-05,2025,Benfica,Fora,44%,0.05,0,1,1,0,0,0,199,0,157,0,12,1,3,57%,1,0
287,2025-09-30,2025,Benfica,Fora,34%,0.47,0,4,2,3,1,1,172,0,141,0,12,0,6,42%,1,0
286,2025-09-26,2025,Benfica,Casa,49%,1.19,2,7,3,4,0,0,184,2,148,0,21,2,8,51%,0,0


In [3189]:
jogos_casa = casa.shape[0]

jogos_casa

10

Posse de bola média

In [3190]:
casa["ball_possession"] = casa["ball_possession"].str.replace("%", "", regex=False)
casa["ball_possession"] = pd.to_numeric(casa["ball_possession"])

In [3191]:
posse_bola_casa = round(casa["ball_possession"].mean(), 2)

posse_bola_casa

58.2

XG Médio

In [3192]:
xg_medio_casa = casa.expected_goals.mean().round(2)

xg_medio_casa

0.68

XG Total

In [3193]:
xg_total_casa = casa.expected_goals.sum().round(2)

xg_total_casa

6.84

Grandes chances criadas

In [3194]:
grandes_chances_criadas_casa = casa.big_chances.sum()

grandes_chances_criadas_casa

14

Média de grandes chances por jogo

In [3195]:
grandes_chances_jogo_casa = round(grandes_chances_criadas_casa / jogos_casa, 2)

grandes_chances_jogo_casa

1.4

Grandes chances convertidas

In [3196]:
grandes_chances_convertidas_casa = casa.big_chances_scored.sum()

grandes_chances_convertidas_casa

8

Aproveitamento das grandes chances

In [3197]:
aproveitamento_grandes_chances_casa = round((grandes_chances_convertidas_casa / grandes_chances_criadas_casa) * 100, 2)

aproveitamento_grandes_chances_casa

57.14

Finalizações

In [3198]:
finalizacoes_casa = casa.total_shots.sum()

finalizacoes_casa

66

Média de finalizações por jogo

In [3199]:
finalizacoes_jogo_casa = round(finalizacoes_casa / jogos_casa, 2)

finalizacoes_jogo_casa

6.6

Finalizações certas

In [3200]:
finalizacoes_certas_casa = casa.shots_on_target.sum()

finalizacoes_certas_casa

26

Finaizações certas por jogo

In [3201]:
finalizacoes_certas_jogo_casa = round(finalizacoes_certas_casa / jogos_casa, 2)

finalizacoes_certas_jogo_casa

2.6

Aproveitamento das finalizações

In [3202]:
aproveitamento_finalizacoes_casa = round((finalizacoes_certas_casa / finalizacoes_casa) * 100, 2)

aproveitamento_finalizacoes_casa

39.39

Finalizações dentro da área

In [3203]:
finalizacoes_area_casa = casa.shots_inside_box.sum()

finalizacoes_area_casa

42

Finalizações dentro da área por jogo

In [3204]:
finalizacoes_area_jogo_casa = round(finalizacoes_area_casa / jogos_casa, 2)

finalizacoes_area_jogo_casa

4.2

Finalizações dentro da área em relação ao total de finalizações

In [3205]:
perc_finalizacoes_area_casa = round((finalizacoes_area_casa / finalizacoes_casa) * 100, 2)

perc_finalizacoes_area_casa

63.64

Bolas na trave

In [3206]:
bolas_trave_casa = casa.hit_woodwork.sum()

bolas_trave_casa

1

Escanteios

In [3207]:
escanteios_casa = casa.corner_kicks.sum()

escanteios_casa

20

Escanteios por jogo

In [3208]:
escanteios_jogo_casa = round(escanteios_casa / jogos_casa, 2)

escanteios_jogo_casa

2.0

Passes

In [3209]:
passes_casa = casa.passes.sum()

passes_casa

2572

Passes certos

In [3210]:
passes_certos_casa = casa.accurate_passes.sum()

passes_certos_casa

2228

Aproveitamento nos passes

In [3211]:
perc_passes_certos_casa = round((passes_certos_casa / passes_casa) * 100, 2)

perc_passes_certos_casa

86.63

Entradas no terço final

In [3212]:
emtradas_terco_final_casa = casa.final_third_entries.sum()

emtradas_terco_final_casa

300

Entradas no terço final por jogo

In [3213]:
emtradas_terco_final_jogo_casa = round(emtradas_terco_final_casa / jogos_casa, 2)

emtradas_terco_final_jogo_casa

30.0

Faltas sofridas no terço final

In [3214]:
faltas_sofridas_terco_final_casa = casa.fouled_in_final_third.sum()

faltas_sofridas_terco_final_casa

11

Perda da posse de bola

In [3215]:
perda_bola_casa = casa.dispossessed.sum()

perda_bola_casa

47

Perda da posse de bola por jogo

In [3216]:
perda_bola_jogo_casa = round(perda_bola_casa / jogos_casa, 2)

perda_bola_jogo_casa

4.7

Duelos ganhos

In [3217]:
casa["duels"] = casa["duels"].str.replace("%", "", regex=False).astype(float)

In [3218]:
duelos_ganhos_casa = casa.duels.mean().round(2)

duelos_ganhos_casa

49.8

Cartões amarelos

In [3219]:
amarelos_primeiro_tempo_casa = casa.yellow_cards.sum()

amarelos_primeiro_tempo_casa

6

Cartões vermelhos

In [3220]:
vermelhos_primeiro_tempo_casa = casa.red_cards.sum()

vermelhos_primeiro_tempo_casa

0

Salvando a informação

In [3221]:
dados = {
    "Jogos": jogos_casa,
    "Posse de bola (média)": posse_bola_casa,
    "xG médio": xg_medio_casa,
    "xG total": xg_total_casa,
    "Grandes chances criadas": grandes_chances_criadas_casa,
    "Grandes chances por jogo": grandes_chances_jogo_casa,
    "Grandes chances convertidas": grandes_chances_convertidas_casa,
    "Aproveitamento das grandes chances": aproveitamento_grandes_chances_casa,
    "Finalizações": finalizacoes_casa,
    "Finalizações por jogo": finalizacoes_jogo_casa,
    "Finalizações certas": finalizacoes_certas_casa,
    "Finalizações certas por jogo": finalizacoes_certas_jogo_casa,
    "% Acerto nas finalizações:": aproveitamento_finalizacoes_casa,
    "Finalizações dentro da área": finalizacoes_area_casa,
    "Finalizações dentro da área por jogo": finalizacoes_area_jogo_casa,
    "% de finalizações dentro da área": perc_finalizacoes_area_casa,
    "Bolas na trave": bolas_trave_casa,
    "Escanteios": escanteios_casa,
    "Escanteios por jogo": escanteios_jogo_casa,
    "Passes": passes_casa,
    "Passes certos": passes_certos_casa,
    "Assertividade nos passes": perc_passes_certos_casa,
    "Entradas no terço final": emtradas_terco_final_casa,
    "Entradas no terço final por jogo": emtradas_terco_final_jogo_casa,
    "Faltas sofridas no terço final": faltas_sofridas_terco_final_casa,
    "Perdas de bola": perda_bola_casa,
    "Perdas de bola por jogo": perda_bola_jogo_casa,
    "Duelos ganhos (média)": duelos_ganhos_casa,
    "Cartões amarelos": amarelos_primeiro_tempo_casa,
    "Cartões vermelhos": vermelhos_primeiro_tempo_casa
}

In [3222]:
ordem = [
    "Jogos", "Posse de bola (média)", "xG médio", "xG total",
    "Grandes chances criadas", "Grandes chances por jogo",
    "Grandes chances convertidas", "Aproveitamento das grandes chances",
    "Finalizações", "Finalizações por jogo", "Finalizações certas",
    "Finalizações certas por jogo", "% Acerto nas finalizações:",
    "Finalizações dentro da área", "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área", "Bolas na trave", "Escanteios", "Escanteios por jogo",
    "Passes", "Passes certos", "Assertividade nos passes", "Entradas no terço final", "Entradas no terço final por jogo",
    "Faltas sofridas no terço final", "Perdas de bola", "Perdas de bola por jogo", "Duelos ganhos (média)",
    "Cartões amarelos", "Cartões vermelhos"
]

In [3223]:
import unicodedata

def normalizar(s: str) -> str:
    return unicodedata.normalize("NFC", s.strip())

# Normalizar as listas
dados = {normalizar(k): v for k, v in dados.items()}
ordem = [normalizar(k) for k in ordem]


In [3224]:
# --- Checar chaves faltantes ---
faltando = [k for k in ordem if k not in dados]
if faltando:
    print("Debug - Chaves em dados:", list(dados.keys()))
    print("Debug - Chaves em ordem:", ordem)
    raise KeyError(f"As seguintes chaves não estão no dicionário 'dados': {faltando}")

# --- Função auxiliar para extrair valor escalar ---
def extrair_valor(v):
    if isinstance(v, (list, tuple, pd.Series)):
        return v[0]
    if hasattr(v, "iloc"):  # DataFrame
        return v.iloc[0]
    return v

# --- Reordenar o dicionário já com valores escalares ---
dados_ordenados = {k: extrair_valor(dados[k]) for k in ordem}

# --- Criar DataFrame ---
df_final_partidas = pd.DataFrame.from_dict(
    dados_ordenados, orient="index", columns=["_temp"]
)

# --- Variáveis que devem ser float ---
variaveis_float = {
    "Posse de bola (média)",
    "xG médio",
    "xG total",
    "odd média",
    "Aproveitamento das grandes chances",
    "Finalizações por jogo",
    "Finalizações certas por jogo",
    "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área",
    "Entradas no terço final por jogo",
    "Escanteios por jogo",
    "Assertividade nos passes",
    "Perdas de bola por jogo",
    "Duelos ganhos (média)",
}

# --- Função para formatar ---
def _fmt(nome, valor):
    valor = extrair_valor(valor)  # garante escalar
    try:
        if nome in variaveis_float:
            return f"{float(valor):.2f}"
        else:
            return f"{int(valor)}"
    except (ValueError, TypeError):
        # se não der para converter em número, retorna como string
        return str(valor)

# --- Nome do clube ---
coluna_clube = clubes.casa

# Renomeia a coluna temporária para o nome do clube
df_final_partidas = df_final_partidas.rename(columns={"_temp": coluna_clube})

# Aplicar formatação
df_final_partidas[coluna_clube] = [
    _fmt(nome, valor) for nome, valor in zip(df_final_partidas.index, df_final_partidas[coluna_clube])
]

# Índice vira coluna "Informações"
tabela_tempo1 = df_final_partidas.reset_index().rename(columns={"index": "Informações"})

In [3225]:
tabela_tempo1.style.hide(axis="index")

Informações,Benfica
Jogos,10
Posse de bola (média),58.20
xG médio,0.68
xG total,6.84
Grandes chances criadas,14
Grandes chances por jogo,1
Grandes chances convertidas,8
Aproveitamento das grandes chances,57.14
Finalizações,66
Finalizações por jogo,6.60


Salvando o arquivo no Excel

In [3226]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    tabela_tempo1.to_excel(writer, sheet_name="Tempo 1", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


## 2º Tempo

### Carregando a consulta

In [3227]:
conn = mysql.connector.connect(**config)

In [3228]:
query = f"""
SELECT
    p.data,
    t2.temporada,
    t2.clube,
    CASE
        WHEN t2.clube = p.casa THEN 'Casa'
        WHEN t2.clube = p.fora THEN 'Fora'
        ELSE 'Verificar'
    END AS mando,
    t2.ball_possession,
    t2.expected_goals,
    t2.big_chances,
    t2.total_shots,
    t2.shots_on_target,
    t2.shots_inside_box,
    t2.hit_woodwork,
    t2.corner_kicks,
    t2.passes,
    t2.big_chances_scored,
    t2.accurate_passes,
    t2.final_third_entries,
    t2.fouled_in_final_third,
    t2.dispossessed,
    t2.duels,
    t2.yellow_cards,
    t2.red_cards
FROM
    2_tempo t2
    INNER JOIN partidas p ON t2.id_jogo = p.id_jogo
WHERE
    t2.clube IN ("{clubes.casa}");
"""

In [3229]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_35332\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [3230]:
conn.close()

In [3231]:
df = df[df["temporada"] == "2025"]

In [3232]:
df.head()

,data,temporada,clube,mando,ball_possession,expected_goals,big_chances,total_shots,shots_on_target,shots_inside_box,hit_woodwork,corner_kicks,passes,big_chances_scored,accurate_passes,final_third_entries,fouled_in_final_third,dispossessed,duels,yellow_cards,red_cards
0,2025-06-16,2025,Benfica,Fora,63%,0.0,1,5,2,4,0,4,190,1,160,28,0,1,57%,2,1
1,2025-06-20,2025,Benfica,Casa,65%,0.0,6,14,8,9,1,2,286,4,251,43,1,2,55%,2,0
2,2025-06-24,2025,Benfica,Casa,24%,0.0,1,3,1,2,0,0,108,0,74,20,0,1,51%,2,0
3,2025-06-28,2025,Benfica,Casa,48%,0.0,1,3,1,3,0,0,185,1,161,17,3,6,55%,5,0
109,2025-07-31,2025,Benfica,Fora,42%,0.0,0,5,2,2,0,4,160,0,132,28,0,2,54%,5,0


### Casa

Jogos

In [3233]:
# Selecionando apenas o time mandante
casa = df[df["clube"] == clubes.casa]

# Filtrando apenas os últimos 10 jogos
casa = casa.sort_values("data", ascending=False).head(10)

casa.head(3)

,data,temporada,clube,mando,ball_possession,expected_goals,big_chances,total_shots,shots_on_target,shots_inside_box,hit_woodwork,corner_kicks,passes,big_chances_scored,accurate_passes,final_third_entries,fouled_in_final_third,dispossessed,duels,yellow_cards,red_cards
288,2025-10-05,2025,Benfica,Fora,54%,0.09,0,3,0,2,1,3,217,0,175,21,2,2,56%,1,0
287,2025-09-30,2025,Benfica,Fora,55%,0.38,1,5,1,4,0,4,230,0,190,21,0,14,51%,4,0
286,2025-09-26,2025,Benfica,Casa,45%,0.08,0,3,1,0,0,0,174,0,132,21,3,7,59%,3,0


In [3234]:
jogos_casa = casa.shape[0]

jogos_casa

10

Posse de bola média

In [3235]:
casa["ball_possession"] = casa["ball_possession"].str.replace("%", "", regex=False)
casa["ball_possession"] = pd.to_numeric(casa["ball_possession"])

In [3236]:
posse_bola_casa = casa.ball_possession.mean().round(2)

posse_bola_casa

C:\Users\stefa\AppData\Roaming\Python\Python312\site-packages\IPython\core\displayhook.py:281: UserWarning: Output cache limit (currently 1000 entries) hit.
Flushing oldest 200 entries.
  warn('Output cache limit (currently {sz} entries) hit.\n'


56.7

XG Médio

In [3237]:
xg_medio_casa = casa.expected_goals.mean().round(2)

xg_medio_casa

0.74

XG Total

In [3238]:
xg_total_casa = casa.expected_goals.sum().round(2)

xg_total_casa

7.35

Grandes chances criadas

In [3239]:
grandes_chances_criadas_casa = casa.big_chances.sum()

grandes_chances_criadas_casa

13

Média de grandes chances por jogo

In [3240]:
grandes_chances_jogo_casa = round(grandes_chances_criadas_casa / jogos_casa, 2)

grandes_chances_jogo_casa

1.3

Grandes chances convertidas

In [3241]:
grandes_chances_convertidas_casa = casa.big_chances_scored.sum()

grandes_chances_convertidas_casa

5

Aproveitamento das grandes chances

In [3242]:
aproveitamento_grandes_chances_casa = round((grandes_chances_convertidas_casa / grandes_chances_criadas_casa) * 100, 2)

aproveitamento_grandes_chances_casa

38.46

Finalizações

In [3243]:
finalizacoes_casa = casa.total_shots.sum()

finalizacoes_casa

70

Média de finalizações por jogo

In [3244]:
finalizacoes_jogo_casa = round(finalizacoes_casa / jogos_casa, 2)

finalizacoes_jogo_casa

7.0

Finalizações certas

In [3245]:
finalizacoes_certas_casa = casa.shots_on_target.sum()

finalizacoes_certas_casa

22

Finaizações certas por jogo

In [3246]:
finalizacoes_certas_jogo_casa = round(finalizacoes_certas_casa / jogos_casa, 2)

finalizacoes_certas_jogo_casa

2.2

Aproveitamento das finalizações

In [3247]:
aproveitamento_finalizacoes_casa = round((finalizacoes_certas_casa / finalizacoes_casa) * 100, 2)

aproveitamento_finalizacoes_casa

31.43

Finalizações dentro da área

In [3248]:
finalizacoes_area_casa = casa.shots_inside_box.sum()

finalizacoes_area_casa

48

Finalizações dentro da área por jogo

In [3249]:
finalizacoes_area_jogo_casa = round(finalizacoes_area_casa / jogos_casa, 2)

finalizacoes_area_jogo_casa

4.8

Finalizações dentro da área em relação ao total de finalizações

In [3250]:
perc_finalizacoes_area_casa = round((finalizacoes_area_casa / finalizacoes_casa) * 100, 2)

perc_finalizacoes_area_casa

68.57

Bolas na trave

In [3251]:
bolas_trave_casa = casa.hit_woodwork.sum()

bolas_trave_casa

4

Escanteios

In [3252]:
escanteios_casa = casa.corner_kicks.sum()

escanteios_casa

31

Escanteios por jogo

In [3253]:
escanteios_jogo_casa = round(escanteios_casa / jogos_casa, 2)

escanteios_jogo_casa

3.1

Passes

In [3254]:
passes_casa = casa.passes.sum()

passes_casa

2358

Passes certos

In [3255]:
passes_certos_casa = casa.accurate_passes.sum()

passes_certos_casa

1990

Aproveitamento nos passes

In [3256]:
perc_passes_certos_casa = round((passes_certos_casa / passes_casa) * 100, 2)

perc_passes_certos_casa

84.39

Entradas no terço final

In [3257]:
emtradas_terco_final_casa = casa.final_third_entries.sum()

emtradas_terco_final_casa

288

Entradas no terço final por jogo

In [3258]:
emtradas_terco_final_jogo_casa = round(emtradas_terco_final_casa / jogos_casa, 2)

emtradas_terco_final_jogo_casa

28.8

Faltas sofridas no terço final

In [3259]:
faltas_sofridas_terco_final_casa = casa.fouled_in_final_third.sum()

faltas_sofridas_terco_final_casa

19

Perda da posse de bola

In [3260]:
perda_bola_casa = casa.dispossessed.sum()

perda_bola_casa

55

Perda da posse de bola por jogo

In [3261]:
perda_bola_jogo_casa = round(perda_bola_casa / jogos_casa, 2)

perda_bola_jogo_casa

5.5

Duelos ganhos

In [3262]:
casa["duels"] = casa["duels"].str.replace("%", "", regex=False).astype(float)

In [3263]:
duelos_ganhos_casa = casa.duels.mean().round(2)

duelos_ganhos_casa

53.4

Cartões amarelos

In [3264]:
amarelos_segundo_tempo_casa = casa.yellow_cards.sum()

amarelos_segundo_tempo_casa

19

Cartões vermelhos

In [3265]:
vermelhos_segundo_tempo_casa = casa.red_cards.sum()

vermelhos_segundo_tempo_casa

1

Salvando a informação

In [3266]:
dados = {
    "Jogos": jogos_casa,
    "Posse de bola (média)": posse_bola_casa,
    "xG médio": xg_medio_casa,
    "xG total": xg_total_casa,
    "Grandes chances criadas": grandes_chances_criadas_casa,
    "Grandes chances por jogo": grandes_chances_jogo_casa,
    "Grandes chances convertidas": grandes_chances_convertidas_casa,
    "Aproveitamento das grandes chances": aproveitamento_grandes_chances_casa,
    "Finalizações": finalizacoes_casa,
    "Finalizações por jogo": finalizacoes_jogo_casa,
    "Finalizações certas": finalizacoes_certas_casa,
    "Finalizações certas por jogo": finalizacoes_certas_jogo_casa,
    "% Acerto nas finalizações:": aproveitamento_finalizacoes_casa,
    "Finalizações dentro da área": finalizacoes_area_casa,
    "Finalizações dentro da área por jogo": finalizacoes_area_jogo_casa,
    "% de finalizações dentro da área": perc_finalizacoes_area_casa,
    "Bolas na trave": bolas_trave_casa,
    "Escanteios": escanteios_casa,
    "Escanteios por jogo": escanteios_jogo_casa,
    "Passes": passes_casa,
    "Passes certos": passes_certos_casa,
    "Assertividade nos passes": perc_passes_certos_casa,
    "Entradas no terço final": emtradas_terco_final_casa,
    "Entradas no terço final por jogo": emtradas_terco_final_jogo_casa,
    "Faltas sofridas no terço final": faltas_sofridas_terco_final_casa,
    "Perdas de bola": perda_bola_casa,
    "Perdas de bola por jogo": perda_bola_jogo_casa,
    "Duelos ganhos (média)": duelos_ganhos_casa,
    "Cartões amarelos": amarelos_segundo_tempo_casa,
    "Cartões vermelhos": vermelhos_segundo_tempo_casa
}

In [3267]:
ordem = [
    "Jogos", "Posse de bola (média)", "xG médio", "xG total",
    "Grandes chances criadas", "Grandes chances por jogo",
    "Grandes chances convertidas", "Aproveitamento das grandes chances",
    "Finalizações", "Finalizações por jogo", "Finalizações certas",
    "Finalizações certas por jogo", "% Acerto nas finalizações:",
    "Finalizações dentro da área", "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área", "Bolas na trave", "Escanteios", "Escanteios por jogo",
    "Passes", "Passes certos", "Assertividade nos passes", "Entradas no terço final", "Entradas no terço final por jogo",
    "Faltas sofridas no terço final", "Perdas de bola", "Perdas de bola por jogo", "Duelos ganhos (média)",
    "Cartões amarelos", "Cartões vermelhos"
]

In [3268]:
import unicodedata

def normalizar(s: str) -> str:
    return unicodedata.normalize("NFC", s.strip())

# Normalizar as listas
dados = {normalizar(k): v for k, v in dados.items()}
ordem = [normalizar(k) for k in ordem]


In [3269]:
# --- Checar chaves faltantes ---
faltando = [k for k in ordem if k not in dados]
if faltando:
    print("Debug - Chaves em dados:", list(dados.keys()))
    print("Debug - Chaves em ordem:", ordem)
    raise KeyError(f"As seguintes chaves não estão no dicionário 'dados': {faltando}")

# --- Função auxiliar para extrair valor escalar ---
def extrair_valor(v):
    if isinstance(v, (list, tuple, pd.Series)):
        return v[0]
    if hasattr(v, "iloc"):  # DataFrame
        return v.iloc[0]
    return v

# --- Reordenar o dicionário já com valores escalares ---
dados_ordenados = {k: extrair_valor(dados[k]) for k in ordem}

# --- Criar DataFrame ---
df_final_partidas = pd.DataFrame.from_dict(
    dados_ordenados, orient="index", columns=["_temp"]
)

# --- Variáveis que devem ser float ---
variaveis_float = {
    "Posse de bola (média)",
    "xG médio",
    "xG total",
    "odd média",
    "Aproveitamento das grandes chances",
    "Finalizações por jogo",
    "Finalizações certas por jogo",
    "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área",
    "Entradas no terço final por jogo",
    "Escanteios por jogo",
    "Assertividade nos passes",
    "Perdas de bola por jogo",
    "Duelos ganhos (média)",
}

# --- Função para formatar ---
def _fmt(nome, valor):
    valor = extrair_valor(valor)  # garante escalar
    try:
        if nome in variaveis_float:
            return f"{float(valor):.2f}"
        else:
            return f"{int(valor)}"
    except (ValueError, TypeError):
        # se não der para converter em número, retorna como string
        return str(valor)

# --- Nome do clube ---
coluna_clube = clubes.casa

# Renomeia a coluna temporária para o nome do clube
df_final_partidas = df_final_partidas.rename(columns={"_temp": coluna_clube})

# Aplicar formatação
df_final_partidas[coluna_clube] = [
    _fmt(nome, valor) for nome, valor in zip(df_final_partidas.index, df_final_partidas[coluna_clube])
]

# Índice vira coluna "Informações"
tabela_tempo2 = df_final_partidas.reset_index().rename(columns={"index": "Informações"})

In [3270]:
tabela_tempo2.style.hide(axis="index")

Informações,Benfica
Jogos,10
Posse de bola (média),56.70
xG médio,0.74
xG total,7.35
Grandes chances criadas,13
Grandes chances por jogo,1
Grandes chances convertidas,5
Aproveitamento das grandes chances,38.46
Finalizações,70
Finalizações por jogo,7.00


Salvando o arquivo no Excel

In [3271]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    tabela_tempo2.to_excel(writer, sheet_name="Tempo 2", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


## Partida completa

### Carregando a consulta

In [3272]:
conn = mysql.connector.connect(**config)

In [3273]:
query = f"""
SELECT
    p.data,
    ep.temporada,
    ep.clube,
    CASE
        WHEN ep.clube = p.casa THEN 'Casa'
        WHEN ep.clube = p.fora THEN 'Fora'
        ELSE 'Verificar'
    END AS mando,
    ep.ball_possession,
    ep.expected_goals,
    ep.big_chances,
    ep.total_shots,
    ep.shots_on_target,
    ep.shots_inside_box,
    ep.hit_woodwork,
    ep.corner_kicks,
    ep.passes,
    ep.big_chances_scored,
    ep.accurate_passes,
    ep.final_third_entries,
    ep.fouled_in_final_third,
    ep.dispossessed,
    ep.duels,
    ep.yellow_cards
FROM
    estatisticas_partidas ep
    INNER JOIN partidas p ON ep.id_jogo = p.id_jogo
WHERE
    ep.clube IN ("{clubes.casa}");
"""

In [3274]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_35332\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [3275]:
conn.close()

In [3276]:
df = df[df["temporada"] == "2025"]

In [3277]:
df.head()

,data,temporada,clube,mando,ball_possession,expected_goals,big_chances,total_shots,shots_on_target,shots_inside_box,hit_woodwork,corner_kicks,passes,big_chances_scored,accurate_passes,final_third_entries,fouled_in_final_third,dispossessed,duels,yellow_cards
0,2025-06-16,2025,Benfica,Fora,61%,0.0,2,8,3,7,0,7,419,2,353,61,2,8,53%,2
1,2025-06-20,2025,Benfica,Casa,65%,0.0,11,32,16,19,2,7,536,5,473,75,3,4,52%,3
2,2025-06-24,2025,Benfica,Casa,27%,0.0,1,7,4,3,0,3,257,0,198,36,1,6,57%,3
3,2025-06-28,2025,Benfica,Casa,36%,0.0,1,8,5,6,0,2,450,1,379,41,4,11,52%,7
109,2025-07-31,2025,Benfica,Fora,46%,0.0,0,9,2,3,0,7,363,0,304,55,3,6,53%,6


### Casa

In [3278]:
# Selecionando apenas o time de casa
casa = df[df["clube"] == clubes.casa]

# Filtrando apenas os últimos 10 jogos
casa = casa.sort_values("data", ascending=False).head(10)

casa.head(3)

,data,temporada,clube,mando,ball_possession,expected_goals,big_chances,total_shots,shots_on_target,shots_inside_box,hit_woodwork,corner_kicks,passes,big_chances_scored,accurate_passes,final_third_entries,fouled_in_final_third,dispossessed,duels,yellow_cards
288,2025-10-05,2025,Benfica,Fora,48%,0.13,0,4,1,2,1,3,416,0,332,33,3,5,57%,2
287,2025-09-30,2025,Benfica,Fora,44%,0.85,1,9,3,7,1,5,402,0,331,33,0,20,48%,5
286,2025-09-26,2025,Benfica,Casa,47%,1.27,2,10,4,4,0,0,358,2,280,42,5,15,55%,3


In [3279]:
jogos_casa = casa.shape[0]

jogos_casa

10

Posse de bola média

In [3280]:
casa["ball_possession"] = casa["ball_possession"].str.replace("%", "", regex=False)
casa["ball_possession"] = pd.to_numeric(casa["ball_possession"])

In [3281]:
posse_bola_casa = casa.ball_possession.mean().round(2)

posse_bola_casa

57.5

XG Médio

In [3282]:
xg_medio_casa = casa.expected_goals.mean().round(2)

xg_medio_casa

1.42

XG Total

In [3283]:
xg_total_casa = casa.expected_goals.sum().round(2)

xg_total_casa

14.17

Grandes chances criadas

In [3284]:
grandes_chances_criadas_casa = casa.big_chances.sum()

grandes_chances_criadas_casa

27

Média de grandes chances por jogo

In [3285]:
grandes_chances_jogo_casa = round(grandes_chances_criadas_casa / jogos_casa, 2)

grandes_chances_jogo_casa

2.7

Grandes chances convertidas

In [3286]:
grandes_chances_convertidas_casa = casa.big_chances_scored.sum()

grandes_chances_convertidas_casa

13

Aproveitamento das grandes chances

In [3287]:
aproveitamento_grandes_chances_casa = round((grandes_chances_convertidas_casa / grandes_chances_criadas_casa) * 100, 2)

aproveitamento_grandes_chances_casa

48.15

Finalizações


In [3288]:
finalizacoes_casa = casa.total_shots.sum()

finalizacoes_casa

136

Média de finalizações por jogo

In [3289]:
finalizacoes_jogo_casa = round(finalizacoes_casa / jogos_casa, 2)

finalizacoes_jogo_casa

13.6

Finalizações certas

In [3290]:
finalizacoes_certas_casa = casa.shots_on_target.sum()

finalizacoes_certas_casa

48

Finaizações certas por jogo

In [3291]:
finalizacoes_certas_jogo_casa = round(finalizacoes_certas_casa / jogos_casa, 2)

finalizacoes_certas_jogo_casa

4.8

Aproveitamento das finalizações

In [3292]:
aproveitamento_finalizacoes_casa = round((finalizacoes_certas_casa / finalizacoes_casa) * 100, 2)

aproveitamento_finalizacoes_casa

35.29

Finalizações dentro da área

In [3293]:
finalizacoes_area_casa = casa.shots_inside_box.sum()

finalizacoes_area_casa

90

Finalizações dentro da área por jogo

In [3294]:
finalizacoes_area_jogo_casa = round(finalizacoes_area_casa / jogos_casa, 2)

finalizacoes_area_jogo_casa

9.0

Finalizações dentro da área em relação ao total de finalizações

In [3295]:
perc_finalizacoes_area_casa = round((finalizacoes_area_casa / finalizacoes_casa) * 100, 2)

perc_finalizacoes_area_casa

66.18

Bolas na trave

In [3296]:
bolas_trave_casa = casa.hit_woodwork.sum()

bolas_trave_casa

5

Escanteios

In [3297]:
escanteios_casa = casa.corner_kicks.sum()

escanteios_casa

51

Escanteios por jogo

In [3298]:
escanteios_jogo_casa = round(escanteios_casa / jogos_casa, 2)

escanteios_jogo_casa

5.1

Passes

In [3299]:
passes_casa = casa.passes.sum()

passes_casa

4930

Passes certos

In [3300]:
passes_certos_casa = casa.accurate_passes.sum()

passes_certos_casa

4218

Aproveitamento nos passes

In [3301]:
perc_passes_certos_casa = round((passes_certos_casa / passes_casa) * 100, 2)

perc_passes_certos_casa

85.56

Entradas no terço final

In [3302]:
emtradas_terco_final_casa = casa.final_third_entries.sum()

emtradas_terco_final_casa

588

Entradas no terço final

In [3303]:
emtradas_terco_final_casa = casa.final_third_entries.sum()

emtradas_terco_final_casa

588

Entradas no terço final por jogo

In [3304]:
emtradas_terco_final_jogo_casa = round(emtradas_terco_final_casa / jogos_casa, 2)

emtradas_terco_final_jogo_casa

58.8

Faltas sofridas no terço final

In [3305]:
faltas_sofridas_terco_final_casa = casa.fouled_in_final_third.sum()

faltas_sofridas_terco_final_casa

30

Perda da posse de bola

In [3306]:
perda_bola_casa = casa.dispossessed.sum()

perda_bola_casa

102

Perda da posse de bola por jogo

In [3307]:
perda_bola_jogo_casa = round(perda_bola_casa / jogos_casa, 2)

perda_bola_jogo_casa

10.2

Duelos ganhos

In [3308]:
casa["duels"] = casa["duels"].str.replace("%", "", regex=False).astype(float)

In [3309]:
duelos_ganhos_casa = casa.duels.mean().round(2)

duelos_ganhos_casa

51.8

Cartões amarelos

In [3310]:
amarelos_casa = casa.yellow_cards.sum()

amarelos_casa

25

Cartões vermelhos

In [3311]:
vermelhos_casa = vermelhos_primeiro_tempo_casa + vermelhos_segundo_tempo_casa

vermelhos_casa

1

Salvando a informação

In [3312]:
dados = {
    "Jogos": jogos_casa,
    "Posse de bola (média)": posse_bola_casa,
    "xG médio": xg_medio_casa,
    "xG total": xg_total_casa,
    "Grandes chances criadas": grandes_chances_criadas_casa,
    "Grandes chances por jogo": grandes_chances_jogo_casa,
    "Grandes chances convertidas": grandes_chances_convertidas_casa,
    "Aproveitamento das grandes chances": aproveitamento_grandes_chances_casa,
    "Finalizações": finalizacoes_casa,
    "Finalizações por jogo": finalizacoes_jogo_casa,
    "Finalizações certas": finalizacoes_certas_casa,
    "Finalizações certas por jogo": finalizacoes_certas_jogo_casa,
    "% Acerto nas finalizações:": aproveitamento_finalizacoes_casa,
    "Finalizações dentro da área": finalizacoes_area_casa,
    "Finalizações dentro da área por jogo": finalizacoes_area_jogo_casa,
    "% de finalizações dentro da área": perc_finalizacoes_area_casa,
    "Bolas na trave": bolas_trave_casa,
    "Escanteios": escanteios_casa,
    "Escanteios por jogo": escanteios_jogo_casa,
    "Passes": passes_casa,
    "Passes certos": passes_certos_casa,
    "Assertividade nos passes": perc_passes_certos_casa,
    "Entradas no terço final": emtradas_terco_final_casa,
    "Entradas no terço final por jogo": emtradas_terco_final_jogo_casa,
    "Faltas sofridas no terço final": faltas_sofridas_terco_final_casa,
    "Perdas de bola": perda_bola_casa,
    "Perdas de bola por jogo": perda_bola_jogo_casa,
    "Duelos ganhos (média)": duelos_ganhos_casa,
    "Cartões amarelos": amarelos_casa,
    "Cartões vermelhos": vermelhos_casa
}

In [3313]:
ordem = [
    "Jogos", "Posse de bola (média)", "xG médio", "xG total",
    "Grandes chances criadas", "Grandes chances por jogo",
    "Grandes chances convertidas", "Aproveitamento das grandes chances",
    "Finalizações", "Finalizações por jogo", "Finalizações certas",
    "Finalizações certas por jogo", "% Acerto nas finalizações:",
    "Finalizações dentro da área", "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área", "Bolas na trave", "Escanteios", "Escanteios por jogo",
    "Passes", "Passes certos", "Assertividade nos passes", "Entradas no terço final", "Entradas no terço final por jogo",
    "Faltas sofridas no terço final", "Perdas de bola", "Perdas de bola por jogo", "Duelos ganhos (média)",
    "Cartões amarelos", "Cartões vermelhos"
]

In [3314]:
import unicodedata

def normalizar(s: str) -> str:
    return unicodedata.normalize("NFC", s.strip())

# Normalizar as listas
dados = {normalizar(k): v for k, v in dados.items()}
ordem = [normalizar(k) for k in ordem]


In [3315]:
# --- Checar chaves faltantes ---
faltando = [k for k in ordem if k not in dados]
if faltando:
    print("Debug - Chaves em dados:", list(dados.keys()))
    print("Debug - Chaves em ordem:", ordem)
    raise KeyError(f"As seguintes chaves não estão no dicionário 'dados': {faltando}")

# --- Função auxiliar para extrair valor escalar ---
def extrair_valor(v):
    if isinstance(v, (list, tuple, pd.Series)):
        return v[0]
    if hasattr(v, "iloc"):  # DataFrame
        return v.iloc[0]
    return v

# --- Reordenar o dicionário já com valores escalares ---
dados_ordenados = {k: extrair_valor(dados[k]) for k in ordem}

# --- Criar DataFrame ---
df_final_partidas = pd.DataFrame.from_dict(
    dados_ordenados, orient="index", columns=["_temp"]
)

# --- Variáveis que devem ser float ---
variaveis_float = {
    "Posse de bola (média)",
    "xG médio",
    "xG total",
    "odd média",
    "Aproveitamento das grandes chances",
    "Finalizações por jogo",
    "Finalizações certas por jogo",
    "Finalizações dentro da área por jogo",
    "% de finalizações dentro da área",
    "Entradas no terço final por jogo",
    "Escanteios por jogo",
    "Assertividade nos passes",
    "Perdas de bola por jogo",
    "Duelos ganhos (média)",
}

# --- Função para formatar ---
def _fmt(nome, valor):
    valor = extrair_valor(valor)  # garante escalar
    try:
        if nome in variaveis_float:
            return f"{float(valor):.2f}"
        else:
            return f"{int(valor)}"
    except (ValueError, TypeError):
        # se não der para converter em número, retorna como string
        return str(valor)

# --- Nome do clube ---
coluna_clube = clubes.casa

# Renomeia a coluna temporária para o nome do clube
df_final_partidas = df_final_partidas.rename(columns={"_temp": coluna_clube})

# Aplicar formatação
df_final_partidas[coluna_clube] = [
    _fmt(nome, valor) for nome, valor in zip(df_final_partidas.index, df_final_partidas[coluna_clube])
]

# Índice vira coluna "Informações"
tabela_completo = df_final_partidas.reset_index().rename(columns={"index": "Informações"})

In [3316]:
tabela_completo.style.hide(axis="index")

Informações,Benfica
Jogos,10
Posse de bola (média),57.50
xG médio,1.42
xG total,14.17
Grandes chances criadas,27
Grandes chances por jogo,2
Grandes chances convertidas,13
Aproveitamento das grandes chances,48.15
Finalizações,136
Finalizações por jogo,13.60


Salvando o arquivo no Excel

In [3317]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    tabela_completo.to_excel(writer, sheet_name="Estatísticas Partidas", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


## Jogadores

### Configurando o banco de dados e carregando a consulta

In [3318]:
conn = mysql.connector.connect(**config)

In [3319]:
query = f"""
SELECT
    p.data,
    p.id_jogo,
    ej.temporada,
    CASE
        WHEN ej.team = p.casa THEN 'Casa'
        WHEN ej.team = p.fora THEN 'Fora'
        ELSE 'Verificar'
    END AS mando,
    ej.player_id,
    ej.name,
    CASE
        WHEN ej.position = 'G' THEN 'Goleiro'
        WHEN ej.position = 'D' THEN 'Defensor'
        WHEN ej.position = 'M' THEN 'Meio-campo'
        WHEN ej.position = 'F' THEN 'Atacante'
        ELSE 'Verificar'
    END AS position,
    ej.team,
    ej.rating,
    ej.minutes_played,
    ej.goals,
    ej.goal_assist,
    ej.big_chance_created,
    ej.key_pass,
    ej.expected_goals
FROM
    estatisticas_jogadores ej
    INNER JOIN partidas p ON ej.id_jogo = p.id_jogo
WHERE
    ej.minutes_played > 0
    AND team IN ('{clubes.casa}');
"""

In [3320]:
df = pd.read_sql(query, conn)

C:\Users\stefa\AppData\Local\Temp\ipykernel_35332\1168034203.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [3321]:
conn.close()

### Mais minutos

Criando uma cópia do DataFrame original

In [3322]:
# Filtrando apenas a temporada atual
df = df[df["temporada"] == "2025"]

df.head(3)

,data,id_jogo,temporada,mando,player_id,name,position,team,rating,minutes_played,goals,goal_assist,big_chance_created,key_pass,expected_goals
0,2025-06-16,13200247,2025,Fora,906800,Leandro Barreiro,Meio-campo,Benfica,0.0,1,0,0,0,0,0.0
1,2025-06-16,13200247,2025,Fora,1201395,Gianluca Prestianni,Meio-campo,Benfica,6.5,13,0,0,0,0,0.0
2,2025-06-16,13200247,2025,Fora,903324,Kerem Aktürkoğlu,Atacante,Benfica,6.6,29,0,0,0,0,0.0


In [3323]:
# Criando a cópia do DataFrame
mais_minutos = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time+jogo"

In [3324]:
game_dates = (
    mais_minutos
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [3325]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [3326]:
mais_minutos = (
    mais_minutos
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores e somando os minutos

In [3327]:
mais_minutos = (
    mais_minutos
      .groupby(["player_id", "name", "position", "team"], as_index=False)["minutes_played"]
      .sum()
      .rename(columns={"minutes_played": "minutos"})
      .sort_values(["team", "minutos"], ascending=[True, False])
      .reset_index(drop=True))

In [3328]:
mais_minutos.head(3)

,player_id,name,position,team,minutos
0,74915,Nicolás Otamendi,Defensor,Benfica,900
1,228364,Fredrik Aursnes,Meio-campo,Benfica,875
2,1015292,Richard Ríos,Meio-campo,Benfica,832


Montando o top 5 de cada clube

In [3329]:
mais_minutos_casa = mais_minutos[mais_minutos["team"] == clubes.casa].head(5)

mais_minutos_casa

,player_id,name,position,team,minutos
0,74915,Nicolás Otamendi,Defensor,Benfica,900
1,228364,Fredrik Aursnes,Meio-campo,Benfica,875
2,1015292,Richard Ríos,Meio-campo,Benfica,832
3,795236,Vangelis Pavlidis,Atacante,Benfica,809
4,993101,Samuel Dahl,Defensor,Benfica,797


In [3330]:
mais_minutos_fora = mais_minutos[mais_minutos["team"] == clubes.fora].head(5)

mais_minutos_fora

,player_id,name,position,team,minutos


Unificando as informações em um único DataFrame

In [3331]:
mais_minutos = pd.concat([mais_minutos_casa, mais_minutos_fora], ignore_index=True).sort_values("minutos", ascending=False)

mais_minutos

,player_id,name,position,team,minutos
0,74915,Nicolás Otamendi,Defensor,Benfica,900
1,228364,Fredrik Aursnes,Meio-campo,Benfica,875
2,1015292,Richard Ríos,Meio-campo,Benfica,832
3,795236,Vangelis Pavlidis,Atacante,Benfica,809
4,993101,Samuel Dahl,Defensor,Benfica,797


### Resultado: mais minutos

Eliminando a primeira coluna

In [3332]:
mais_minutos = mais_minutos.drop(mais_minutos.columns[0], axis=1)

mais_minutos

,name,position,team,minutos
0,Nicolás Otamendi,Defensor,Benfica,900
1,Fredrik Aursnes,Meio-campo,Benfica,875
2,Richard Ríos,Meio-campo,Benfica,832
3,Vangelis Pavlidis,Atacante,Benfica,809
4,Samuel Dahl,Defensor,Benfica,797


Salvando no Excel

In [3333]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    mais_minutos.head(10).to_excel(writer, sheet_name="Mais Minutos", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Artilharia

In [3334]:
gols = df.copy()

Selecionando as colunas de trabalho

In [3335]:
gols = gols[["data", "id_jogo", "player_id", "name", "position", "team", "goals"]]

gols.head()

,data,id_jogo,player_id,name,position,team,goals
0,2025-06-16,13200247,906800,Leandro Barreiro,Meio-campo,Benfica,0
1,2025-06-16,13200247,1201395,Gianluca Prestianni,Meio-campo,Benfica,0
2,2025-06-16,13200247,903324,Kerem Aktürkoğlu,Atacante,Benfica,0
3,2025-06-16,13200247,907461,Orkun Kökçü,Meio-campo,Benfica,0
4,2025-06-16,13200247,220223,Andrea Belotti,Atacante,Benfica,0


Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [3336]:
game_dates = (
    gols
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [3337]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [3338]:
gols = (
    gols
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores e somando os minutos

In [3339]:
gols = (
    gols
      .groupby(["player_id", "name", "position", "team"], as_index=False)["goals"]
      .sum()
      .rename(columns={"goals": "gols"})
      .sort_values(["team", "gols"], ascending=[True, False])
      .reset_index(drop=True))

gols.head(3)

,player_id,name,position,team,gols
0,795236,Vangelis Pavlidis,Atacante,Benfica,5
1,1103333,Franjo Ivanović,Atacante,Benfica,2
2,1123925,Georgiy Sudakov,Meio-campo,Benfica,2


In [3340]:
gols = gols[gols["gols"] > 0]

gols.head(3)

,player_id,name,position,team,gols
0,795236,Vangelis Pavlidis,Atacante,Benfica,5
1,1103333,Franjo Ivanović,Atacante,Benfica,2
2,1123925,Georgiy Sudakov,Meio-campo,Benfica,2


Removendo os valores duplicados

In [3341]:
gols = gols.drop_duplicates()

### Resultado: Artilharia

Ordenando do maior para o menor valor

In [3342]:
gols = gols.sort_values("gols", ascending=False)

gols = gols.head(10)

gols

,player_id,name,position,team,gols
0,795236,Vangelis Pavlidis,Atacante,Benfica,5
1,1103333,Franjo Ivanović,Atacante,Benfica,2
2,1123925,Georgiy Sudakov,Meio-campo,Benfica,2
3,228364,Fredrik Aursnes,Meio-campo,Benfica,1
4,903324,Kerem Aktürkoğlu,Atacante,Benfica,1
5,1087316,Enzo Barrenechea,Meio-campo,Benfica,1
6,1100952,Andreas Schjelderup,Atacante,Benfica,1
7,1102791,Amar Dedić,Defensor,Benfica,1
8,1201395,Gianluca Prestianni,Meio-campo,Benfica,1


Eliminando a coluna "player_id"

In [3343]:
gols = gols.drop(gols.columns[0], axis=1)

gols.head(3)

,name,position,team,gols
0,Vangelis Pavlidis,Atacante,Benfica,5
1,Franjo Ivanović,Atacante,Benfica,2
2,Georgiy Sudakov,Meio-campo,Benfica,2


Salvando as informações

In [3344]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    gols.head(10).to_excel(writer, sheet_name="Artilharia", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Assistências

Criando uma cópia do DataFrame original

In [3345]:
assistencias = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [3346]:
game_dates = (
    assistencias
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [3347]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [3348]:
assistencias = (
    assistencias
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [3349]:
assistencias = (
    assistencias
      .groupby(["player_id", "name", "position", "team"], as_index=False)["goal_assist"]
      .sum()
      .rename(columns={"goal_assist": "assistencias"})
      .sort_values(["team", "assistencias"], ascending=[True, False])
      .reset_index(drop=True))

Eliminando os jogadores que não deram nenhuma assistência

In [3350]:
assistencias = assistencias[assistencias["assistencias"] > 0]

Removendo os valores duplicados

In [3351]:
assistencias = assistencias.drop_duplicates()

### Resultado: Assistências

Ordenando do maior para o menor

In [3352]:
assistencias = assistencias.sort_values("assistencias", ascending=False)

assistencias = assistencias.head(10)

assistencias

,player_id,name,position,team,assistencias
0,795236,Vangelis Pavlidis,Atacante,Benfica,2
1,906800,Leandro Barreiro,Meio-campo,Benfica,2
2,823631,Dodi Lukebakio,Meio-campo,Benfica,1
3,1100952,Andreas Schjelderup,Atacante,Benfica,1
4,1102791,Amar Dedić,Defensor,Benfica,1
5,1123925,Georgiy Sudakov,Meio-campo,Benfica,1


Eliminando a coluna "player_id"

In [3353]:
assistencias = assistencias.drop(assistencias.columns[0], axis=1)

assistencias.head(3)

,name,position,team,assistencias
0,Vangelis Pavlidis,Atacante,Benfica,2
1,Leandro Barreiro,Meio-campo,Benfica,2
2,Dodi Lukebakio,Meio-campo,Benfica,1


Salvando as informações

In [3354]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    assistencias.head(10).to_excel(writer, sheet_name="Assistências", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Participação em gols

In [3355]:
participacao_gols = df.copy()

Criando a coluna com a participação em gols

In [3356]:
participacao_gols["participacoes"] = participacao_gols["goals"] + participacao_gols["goal_assist"]

participacao_gols.head(3)

,data,id_jogo,temporada,mando,player_id,name,position,team,rating,minutes_played,goals,goal_assist,big_chance_created,key_pass,expected_goals,participacoes
0,2025-06-16,13200247,2025,Fora,906800,Leandro Barreiro,Meio-campo,Benfica,0.0,1,0,0,0,0,0.0,0
1,2025-06-16,13200247,2025,Fora,1201395,Gianluca Prestianni,Meio-campo,Benfica,6.5,13,0,0,0,0,0.0,0
2,2025-06-16,13200247,2025,Fora,903324,Kerem Aktürkoğlu,Atacante,Benfica,6.6,29,0,0,0,0,0.0,0


Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [3357]:
game_dates = (
    participacao_gols
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [3358]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [3359]:
participacao_gols = (
    participacao_gols
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [3360]:
participacao_gols = (
    participacao_gols
      .groupby(["player_id", "name", "position", "team"], as_index=False)["participacoes"]
      .sum()
      .sort_values(["team", "participacoes"], ascending=[True, False])
      .reset_index(drop=True))


Removendo os jogadores sem participação em gols

In [3361]:
participacao_gols = participacao_gols[participacao_gols["participacoes"] > 0]

participacao_gols

,player_id,name,position,team,participacoes
0,795236,Vangelis Pavlidis,Atacante,Benfica,7
1,1123925,Georgiy Sudakov,Meio-campo,Benfica,3
2,906800,Leandro Barreiro,Meio-campo,Benfica,2
3,1100952,Andreas Schjelderup,Atacante,Benfica,2
4,1102791,Amar Dedić,Defensor,Benfica,2
5,1103333,Franjo Ivanović,Atacante,Benfica,2
6,228364,Fredrik Aursnes,Meio-campo,Benfica,1
7,823631,Dodi Lukebakio,Meio-campo,Benfica,1
8,903324,Kerem Aktürkoğlu,Atacante,Benfica,1
9,1087316,Enzo Barrenechea,Meio-campo,Benfica,1


### Resultado: Participação em gols

Ordenando do maior para o menor

In [3362]:
participacao_gols = participacao_gols.sort_values("participacoes", ascending=False)

participacao_gols = participacao_gols.head(10)

participacao_gols

,player_id,name,position,team,participacoes
0,795236,Vangelis Pavlidis,Atacante,Benfica,7
1,1123925,Georgiy Sudakov,Meio-campo,Benfica,3
2,906800,Leandro Barreiro,Meio-campo,Benfica,2
3,1100952,Andreas Schjelderup,Atacante,Benfica,2
4,1102791,Amar Dedić,Defensor,Benfica,2
5,1103333,Franjo Ivanović,Atacante,Benfica,2
6,228364,Fredrik Aursnes,Meio-campo,Benfica,1
7,823631,Dodi Lukebakio,Meio-campo,Benfica,1
8,903324,Kerem Aktürkoğlu,Atacante,Benfica,1
9,1087316,Enzo Barrenechea,Meio-campo,Benfica,1


Eliminando a coluna "player_id"

In [3363]:
participacao_gols = participacao_gols.drop(participacao_gols.columns[0], axis=1)

participacao_gols.head(3)

,name,position,team,participacoes
0,Vangelis Pavlidis,Atacante,Benfica,7
1,Georgiy Sudakov,Meio-campo,Benfica,3
2,Leandro Barreiro,Meio-campo,Benfica,2


Salvando as informações

In [3364]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    participacao_gols.head(10).to_excel(writer, sheet_name="Participação em Gols", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Rating Total

In [3365]:
rating = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [3366]:
game_dates = (
    rating
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [3367]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [3368]:
rating = (
    rating
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [3369]:
rating = (
    rating
      .groupby(["player_id", "name", "position", "team"], as_index=False)["rating"]
      .sum()
      .rename(columns={"rating": "rating_total"})
      .sort_values(["team", "rating_total"], ascending=[True, False])
      .reset_index(drop=True))

Montando o top 5 de cada clube

In [3370]:
rating_casa = rating[rating["team"] == clubes.casa].head(5)

rating_casa

,player_id,name,position,team,rating_total
0,74915,Nicolás Otamendi,Defensor,Benfica,73.4
1,228364,Fredrik Aursnes,Meio-campo,Benfica,69.6
2,795236,Vangelis Pavlidis,Atacante,Benfica,68.6
3,993101,Samuel Dahl,Defensor,Benfica,67.8
4,1015292,Richard Ríos,Meio-campo,Benfica,67.5


In [3371]:
rating_fora = rating[rating["team"] == clubes.fora].head(5)

rating_fora

,player_id,name,position,team,rating_total


Unificando as informações em um único DataFrame

In [3372]:
rating = pd.concat([rating_casa, rating_fora], ignore_index=True).sort_values("rating_total", ascending=False)

rating

,player_id,name,position,team,rating_total
0,74915,Nicolás Otamendi,Defensor,Benfica,73.4
1,228364,Fredrik Aursnes,Meio-campo,Benfica,69.6
2,795236,Vangelis Pavlidis,Atacante,Benfica,68.6
3,993101,Samuel Dahl,Defensor,Benfica,67.8
4,1015292,Richard Ríos,Meio-campo,Benfica,67.5


### Resultado: Rating Total

Ordenando do maior para o menor

In [3373]:
rating = rating[rating["rating_total"] > 0]
rating["rating_total"] = rating["rating_total"].map(lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))

rating.head(10).style.hide(axis="index")

player_id,name,position,team,rating_total
74915,Nicolás Otamendi,Defensor,Benfica,"73,40"
228364,Fredrik Aursnes,Meio-campo,Benfica,"69,60"
795236,Vangelis Pavlidis,Atacante,Benfica,"68,60"
993101,Samuel Dahl,Defensor,Benfica,"67,80"
1015292,Richard Ríos,Meio-campo,Benfica,"67,50"


Eliminando a coluna "player_id"

In [3374]:
rating = rating.drop(rating.columns[0], axis=1)

rating

,name,position,team,rating_total
0,Nicolás Otamendi,Defensor,Benfica,"73,40"
1,Fredrik Aursnes,Meio-campo,Benfica,"69,60"
2,Vangelis Pavlidis,Atacante,Benfica,"68,60"
3,Samuel Dahl,Defensor,Benfica,"67,80"
4,Richard Ríos,Meio-campo,Benfica,"67,50"


Salvando as informações

In [3375]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    rating.head(10).to_excel(writer, sheet_name="Rating Total", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Rating Médio

Criando uma cópia do DataFrame original

In [3376]:
rating_medio = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [3377]:
game_dates = (
    rating_medio
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [3378]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [3379]:
rating_medio = (
    rating_medio
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [3380]:
rating_medio = (
    rating_medio
      .groupby(["player_id", "name", "position", "team"], as_index=False)["rating"]
      .mean()
      .rename(columns={"rating": "rating_medio"})
      .sort_values(["team", "rating_medio"], ascending=[True, False])
      .reset_index(drop=True))

#### Criando a coluna de minutos jogados

Criando uma cópia do DataFrame original

In [3381]:
minutos_jogados = df.copy()

Data do jogo por "time + jogo"

In [3382]:
game_dates = (
    minutos_jogados
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [3383]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [3384]:
minutos_jogados = (
    minutos_jogados
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [3385]:
minutos_jogados = (
    minutos_jogados
      .groupby(["player_id", "team"], as_index=False)["minutes_played"]
      .sum()
      .rename(columns={"minutes_played": "minutos_jogados"})
      .sort_values(["team", "minutos_jogados"], ascending=[True, False])
      .reset_index(drop=True))

Transformando os minutos jogados em inteiro

In [3386]:
minutos_jogados["minutos_jogados"] = minutos_jogados["minutos_jogados"].astype(int)

minutos_jogados.head(3)

,player_id,team,minutos_jogados
0,74915,Benfica,900
1,228364,Benfica,875
2,1015292,Benfica,832


Unificando os dois DataFrames

In [3387]:
rating_medio = rating_medio.merge(minutos_jogados, how="inner", on="player_id")

rating_medio.head(3)

,player_id,name,position,team_x,rating_medio,team_y,minutos_jogados
0,983550,Samuel Soares,Goleiro,Benfica,7.50,Benfica,180
1,1087316,Enzo Barrenechea,Meio-campo,Benfica,7.40,Benfica,784
2,74915,Nicolás Otamendi,Defensor,Benfica,7.34,Benfica,900


Filtrando os jogadores com mais de 10% de participação nos jogos

In [3388]:
rating_medio = rating_medio[rating_medio["minutos_jogados"] >= 90]

rating_medio.head(3)

,player_id,name,position,team_x,rating_medio,team_y,minutos_jogados
0,983550,Samuel Soares,Goleiro,Benfica,7.50,Benfica,180
1,1087316,Enzo Barrenechea,Meio-campo,Benfica,7.40,Benfica,784
2,74915,Nicolás Otamendi,Defensor,Benfica,7.34,Benfica,900


Formantando o valor do rating médio com apenas duas casas decimais

In [3389]:
rating_medio["rating_medio"] = rating_medio["rating_medio"].round(2)

rating_medio.head(3)

,player_id,name,position,team_x,rating_medio,team_y,minutos_jogados
0,983550,Samuel Soares,Goleiro,Benfica,7.50,Benfica,180
1,1087316,Enzo Barrenechea,Meio-campo,Benfica,7.40,Benfica,784
2,74915,Nicolás Otamendi,Defensor,Benfica,7.34,Benfica,900


### Resultado: Rating Médio

Ordenando do maior para o menor

In [3390]:
rating_medio = rating_medio[["name", "position", "team_x", "rating_medio"]]
rating_medio = rating_medio.sort_values("rating_medio", ascending=False)
rating_medio["rating_medio"] = rating_medio["rating_medio"].map(lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))

rating_medio.head(10).style.hide(axis="index")

name,position,team_x,rating_medio
Samuel Soares,Goleiro,Benfica,"7,50"
Enzo Barrenechea,Meio-campo,Benfica,"7,40"
Nicolás Otamendi,Defensor,Benfica,"7,34"
Georgiy Sudakov,Meio-campo,Benfica,"7,30"
Dodi Lukebakio,Meio-campo,Benfica,"7,20"
Kerem Aktürkoğlu,Atacante,Benfica,"7,20"
António Silva,Defensor,Benfica,"7,18"
Anatoliy Trubin,Goleiro,Benfica,"7,09"
Gianluca Prestianni,Meio-campo,Benfica,"7,02"
Amar Dedić,Defensor,Benfica,"6,99"


In [3391]:
rating_medio = rating_medio.head(10)

Salvando as informações

In [3392]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    rating_medio.head(10).to_excel(writer, sheet_name="Rating Médio", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Grandes Chances Criadas

Criando uma cópia do DataFrame principal

In [3393]:
grandes_chances = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [3394]:
game_dates = (
    grandes_chances
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [3395]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [3396]:
grandes_chances = (
    grandes_chances
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [3397]:
grandes_chances.head(2)

,data,id_jogo,temporada,mando,player_id,name,position,team,rating,minutes_played,goals,goal_assist,big_chance_created,key_pass,expected_goals
0,2025-09-12,14131901,2025,Casa,998870,Henrique Araújo,Atacante,Benfica,6.5,18,0,0,0,0,0.0000
1,2025-09-12,14131901,2025,Casa,906800,Leandro Barreiro,Meio-campo,Benfica,6.8,18,0,0,0,1,0.0275


In [3398]:
grandes_chances = (
    grandes_chances
      .groupby(["player_id", "name", "position", "team"], as_index=False)["big_chance_created"]
      .sum()
      .rename(columns={"big_chance_created": "grandes_chances"})
      .sort_values(["team", "grandes_chances"], ascending=[True, False])
      .reset_index(drop=True))

Eliminado os jogadores que não criaram nenhuma grande chance

In [3399]:
grandes_chances = grandes_chances[grandes_chances["grandes_chances"] > 0]

grandes_chances.head(3)

,player_id,name,position,team,grandes_chances
0,795236,Vangelis Pavlidis,Atacante,Benfica,3
1,993101,Samuel Dahl,Defensor,Benfica,3
2,228364,Fredrik Aursnes,Meio-campo,Benfica,2


### Resultado: Grandes Chances Criadas

In [3400]:
grandes_chances = grandes_chances.sort_values("grandes_chances", ascending=False)

grandes_chances.head(3)

,player_id,name,position,team,grandes_chances
0,795236,Vangelis Pavlidis,Atacante,Benfica,3
1,993101,Samuel Dahl,Defensor,Benfica,3
2,228364,Fredrik Aursnes,Meio-campo,Benfica,2


Eliminando a coluna "player_id"

In [3401]:
grandes_chances = grandes_chances.drop(grandes_chances.columns[0], axis=1)

grandes_chances.head(3)

,name,position,team,grandes_chances
0,Vangelis Pavlidis,Atacante,Benfica,3
1,Samuel Dahl,Defensor,Benfica,3
2,Fredrik Aursnes,Meio-campo,Benfica,2


In [3402]:
grandes_chances = grandes_chances.head(10)

Salvando as informações

In [3403]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    grandes_chances.head(10).to_excel(writer, sheet_name="Grandes Chances", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### Passes chave

Criando uma cópia do DataFrame original

In [3404]:
passes_chave = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [3405]:
game_dates = (
    passes_chave
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [3406]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [3407]:
passes_chave = (
    passes_chave
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [3408]:
passes_chave = (
    passes_chave
      .groupby(["player_id", "name", "position", "team"], as_index=False)["key_pass"]
      .sum()
      .rename(columns={"key_pass": "passes_chave"})
      .sort_values(["team", "passes_chave"], ascending=[True, False])
      .reset_index(drop=True))

Removendo os jogadores que não deram nenhum passe-chave

In [3409]:
passes_chave = passes_chave[passes_chave["passes_chave"] > 0]

passes_chave.head(3)

,player_id,name,position,team,passes_chave
0,795236,Vangelis Pavlidis,Atacante,Benfica,14
1,228364,Fredrik Aursnes,Meio-campo,Benfica,12
2,993101,Samuel Dahl,Defensor,Benfica,12


### Resultado: Passes Chave

In [3410]:
passes_chave = passes_chave.sort_values("passes_chave", ascending=False)

passes_chave.head(3)

,player_id,name,position,team,passes_chave
0,795236,Vangelis Pavlidis,Atacante,Benfica,14
1,228364,Fredrik Aursnes,Meio-campo,Benfica,12
2,993101,Samuel Dahl,Defensor,Benfica,12


Eliminando a coluna "player_id"

In [3411]:
passes_chave = passes_chave.drop(passes_chave.columns[0], axis=1)

passes_chave.head(3)

,name,position,team,passes_chave
0,Vangelis Pavlidis,Atacante,Benfica,14
1,Fredrik Aursnes,Meio-campo,Benfica,12
2,Samuel Dahl,Defensor,Benfica,12


In [3412]:
passes_chave = passes_chave.head(10)

Salvando as informações

In [3413]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    passes_chave.to_excel(writer, sheet_name="Passes Chave", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### XG Total

In [3414]:
xg = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [3415]:
game_dates = (
    xg
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [3416]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [3417]:
xg = (
    xg
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [3418]:
xg = (
    xg
      .groupby(["player_id", "name", "position", "team"], as_index=False)["expected_goals"]
      .sum()
      .rename(columns={"expected_goals": "xg_total"})
      .sort_values(["team", "xg_total"], ascending=[True, False])
      .reset_index(drop=True))

Eliminando os valores igual a 0

In [3419]:
xg = xg[xg["xg_total"] > 0]

xg.head(3)

,player_id,name,position,team,xg_total
0,795236,Vangelis Pavlidis,Atacante,Benfica,5.2727
1,1103333,Franjo Ivanović,Atacante,Benfica,2.0497
2,1015292,Richard Ríos,Meio-campo,Benfica,1.9593


### Resultado: XG Total

In [3420]:
xg = xg.sort_values("xg_total", ascending=False)
xg["xg_total"] = xg["xg_total"].map(lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))

xg.head(3).style.hide(axis="index")

player_id,name,position,team,xg_total
795236,Vangelis Pavlidis,Atacante,Benfica,"5,27"
1103333,Franjo Ivanović,Atacante,Benfica,"2,05"
1015292,Richard Ríos,Meio-campo,Benfica,"1,96"


Eliminando a coluna "player_id"

In [3421]:
xg = xg.drop(xg.columns[0], axis=1)

xg.head(3)

,name,position,team,xg_total
0,Vangelis Pavlidis,Atacante,Benfica,"5,27"
1,Franjo Ivanović,Atacante,Benfica,"2,05"
2,Richard Ríos,Meio-campo,Benfica,"1,96"


In [3422]:
xg = xg.head(10)

Salvando as informações

In [3423]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    xg.to_excel(writer, sheet_name="xG Total", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


### XG Médio


Criando uma cópia do DataFrame original

In [3424]:
xg_medio = df.copy()

Selecionando os últimos 10 jogos de cada clube

Data do jogo por "time + jogo"

In [3425]:
game_dates = (
    xg_medio
      .groupby(["team", "id_jogo"], as_index=False)["data"]
      .max()
      .rename(columns={"data": "data_jogo"}))

Selecionando os 10 jogos mais recentes por time

In [3426]:
top10_por_time = (
    game_dates
      .sort_values(["team", "data_jogo"], ascending=[True, False])
      .groupby("team", group_keys=False)
      .head(10)[["team", "id_jogo"]])

Filtrando o DF original para manter somente esses jogos

In [3427]:
xg_medio = (
    xg_medio
      .merge(top10_por_time, on=["team", "id_jogo"], how="inner"))

Unificando os jogadores

In [3428]:
xg_medio = (
    xg_medio
      .groupby(["player_id", "name", "position", "team"], as_index=False)["expected_goals"]
      .mean()
      .rename(columns={"expected_goals": "xg_medio"})
      .sort_values(["team", "xg_medio"], ascending=[True, False])
      .reset_index(drop=True))


Removendo os valores igual a 0

In [3429]:
xg_medio = xg_medio[xg_medio["xg_medio"] > 0]

xg_medio.head(3)

,player_id,name,position,team,xg_medio
0,795236,Vangelis Pavlidis,Atacante,Benfica,0.527270
1,823631,Dodi Lukebakio,Meio-campo,Benfica,0.242450
2,1103333,Franjo Ivanović,Atacante,Benfica,0.227744


### Resultado: XG Médio

In [3430]:
xg_medio = xg_medio.sort_values("xg_medio", ascending=False)
xg_medio["xg_medio"] = xg_medio["xg_medio"].map(lambda x: f"{x:,.2f}".replace(",", "X").replace(".", ",").replace("X", "."))

xg_medio.head(3).style.hide(axis="index")

player_id,name,position,team,xg_medio
795236,Vangelis Pavlidis,Atacante,Benfica,"0,53"
823631,Dodi Lukebakio,Meio-campo,Benfica,"0,24"
1103333,Franjo Ivanović,Atacante,Benfica,"0,23"


Eliminando a coluna "player_id"

In [3431]:
xg_medio = xg_medio.drop(xg_medio.columns[0], axis=1)

xg_medio.head(3)

,name,position,team,xg_medio
0,Vangelis Pavlidis,Atacante,Benfica,"0,53"
1,Dodi Lukebakio,Meio-campo,Benfica,"0,24"
2,Franjo Ivanović,Atacante,Benfica,"0,23"


In [3432]:
xg_medio = xg_medio.head(10)

Salvando o arquivo

In [3433]:
with pd.ExcelWriter(caminho, engine="openpyxl",
                    mode="a" if os.path.exists(caminho) else "w",
                    if_sheet_exists="replace") as writer:
    xg_medio.to_excel(writer, sheet_name="xG Médio", index=False)

print(f"Arquivo salvo em: {caminho}")

Arquivo salvo em: C:\Users\stefa\Desktop\bet_dados\analise_jogos.xlsx


# Prompt

In [ ]:
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn

# =========================================
# OPCIONAIS: deixar a string dos DFs sem cortes/quebras
# =========================================
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 1000)  # evita wrap precoce

# =========================================
# FUNÇÃO: converte objeto em texto sem índice
# - Se for Series, vira DataFrame primeiro
# - Se não existir, retorna string vazia (evita quebrar o f-string)
# =========================================
def to_text_no_index(obj):
    if obj is None:
        return ""
    if isinstance(obj, pd.Series):
        obj = obj.to_frame()
    if isinstance(obj, pd.DataFrame):
        # justify="left" ajuda a manter colunas previsíveis
        return obj.to_string(index=False, justify="left")
    return str(obj)

# =========================================
# PEGAR TODOS OS DFs CITADOS E CONVERTER
# (se algum não existir no seu ambiente, vira "")
# =========================================
globals_dict = globals()
names = [
    "tabela", "tabela_recente", "tabela_tempo1", "tabela_tempo2",
    "tabela_completo", "df_cleans", "desempenho_casa", "desempenho_fora",
    "desempenho_confronto", "historico_recente", "mais_minutos", "gols", "assistencias",
    "participacao_gols", "rating", "rating_medio", "grandes_chances",
    "passes_chave", "xg", "xg_medio"
]

conv = {}
for n in names:
    conv[n] = to_text_no_index(globals_dict.get(n))

# =========================================
# AQUI ASSUMO QUE VOCÊ JÁ TEM O OBJETO `clubes`
# com atributos .casa, .fora, .campeonato, .data, .hora
# =========================================

texto = f"""[ PAPEL ] 

Atue como Designer e Editor de Dados Sênior. Crie uma apresentação tipo e-book para a Bet Dados no Gamma (tema: Coal).

[ CONTEXTO ]

Público leigo–intermediário; foco em clareza.

Use somente os dados fornecidos. Sem inferências, estimativas ou previsões.

Se um dado não existir, omite a seção.

Duelo no formato “Time A x Time B” (nunca “vs”).

Estética “big tech”: precisão, consistência, espaço em branco e alinhamento milimétrico.

[ ESTILO GLOBAL — CONTRASTE OBRIGATÓRIO ] Forçar cor do texto = #FFFFFF para qualquer elemento textual, sem exceções:

Aplica cor #FFFFFF com opacidade 100% para: títulos, subtítulos, parágrafos, bullets, textos de cartões, subtítulos/captions, notas, footers, botões, placeholders, links, números de ordenação (ex.: “01”, “02” no canto), legendas de gráficos, rótulos de eixos, rótulos de dados, títulos de tabela e células de tabela.

Proibido usar cinzas/“muted”, opacidade < 100% ou “secondary text”. Nada de “texto desabilitado” em cinza.

Se houver fundo claro/Imagem, coloque placa/overlay escuro (ex.: #0A0A0A 90–95%) antes de posicionar o texto.

Ícones/linhas divisórias podem usar #FFFFFF a 70–90% de opacidade; o texto nunca.

[ GRÁFICOS ]

Desligar cores padrão do tema.

Barras/fatias/linhas: usar apenas cinzas claros #F2F2F2, #EAEAEA, #E0E0E0, #D5D5D5. (Nunca cinzas escuros.)

Contorno das barras/fatias: #FFFFFF (80–100% opacidade) para separação.

Legenda, eixos e labels: #FFFFFF.

Grade discreta: #9E9E9E a 25–35% opacidade. Fundo do gráfico transparente.

Cores oficiais dos clubes apenas como acento (borda/marker), mantendo contraste AA.

[ IDENTIDADE E IMAGENS ]

Escudos oficiais (sem distorção nem marca d’água). Se não houver, usar placeholder neutro.

Proibição absoluta de imagens com marca d’água.

Paleta do deck: #001636 como acento + neutros escuros de fundo + dados em cinzas claros.

[ TAREFA — SLIDES ] (omita os que não tiverem dados)

Capa — Título “[Time A x Time B] | Sub “[Data] — [Hora]”; chips de mandante/visitante e fase; troféu oficial como hero; overlay escuro se preciso.

Publicidade Bet Dados (obrigatório) — texto, CTA “Assinar agora” → https://pay.cakto.com.br/35ww97h_563195 e placeholder de logo.

Na publicidade, frise que a Bet Dados tem dados de mais de 90 mil jogos, mais de 140 campeonatos, competições de mais de 40 países,
mais de 4.200 clubes e mais de 115 mil jogadores. Ressalte também que a Bet Dados é um grupo no Telegram, onde são enviadas análises
dos principais jogos do dia, além de análises especiais com dados de ligas, clubes e/ou jogadores.

Ficha do Jogo — competição, temporada, data/hora, estádio/mandante.

Panorama das Equipes (cards lado a lado, 2–4 linhas por equipe, texto branco).

Indicadores-Chave (tabela/tiles).

Distribuição de Gols por Minutagem.

Ofensivo vs Defensivo.

Bolas Paradas.

Disciplina.

Tendências Temporais.

Head-to-Head (se houver).

Glossário Rápido (somente termos presentes).

Notas Metodológicas (origem/janela/limitações; “sem prognósticos”).

Slide Final (QR/CTA).

[ REGRAS DE CONTEÚDO ]

Sem previsões/odds.

Em hipótese alguma utilize imagens com marca d'água.

Nomes dos clubes exatamente como informados (acentos e caixa).

Percentuais com até 2 casas decimais; unidades PT-BR.

[ ENTRADAS ] Duelo, competição, temporada, data/hora e todas as estatísticas fornecidas (médias, totais, séries, rankings etc.).

[ AUDITORIA DE CONTRASTE (passo obrigatório no final) ]

Varra o deck e liste qualquer elemento cujo texto NÃO esteja #FFFFFF (inclui textos de cartões, subtítulos/captions, numeradores “01/02/03/04”, descrições, botões, rodapés e legendas).

Corrija automaticamente para #FFFFFF e reporte “Corrigido: [elementos]”. Se algo não puder ser alterado, substitua por placa/overlay escuro até garantir contraste AA.

[ CRITÉRIOS DE QUALIDADE ]

Fidelidade 100% aos dados.

Texto 100% branco em todo o deck; gráficos em cinzas claros; contraste AA mínimo.

Coesão (grid, margens, alinhamentos).

Didática (1 frase objetiva por gráfico).

Conformidade de marca (escudos/nome de clubes corretos; zero marca d’água).

[ CHECKLIST FINAL ]

Todo texto branco #FFFFFF (inclui cartões, captions e numeradores “01/02/03/…”).

Séries de gráficos somente #F2F2F2–#D5D5D5 + contorno branco; legenda/eixos brancos.

Sem imagens com marca d’água; escudos/taça oficiais.

“x” no duelo; nomes dos clubes fiéis.

Seções sem dados omitidas; nenhum número inventado.

Exportável para PDF com contraste perfeito.

Não trazer imagens com marca d'água.

Os dados e as estatísticas são de um levantamento da Bet Dados.

NÃO precisa informar a temporada.

Reforçando: NÃO FAÇA NENHUMA PREVISÃO PARA A PARTIDA!

Nos gráficos, SEMPRE ative o rótulo de dados (mostrar valores).

Importante: a Bet Dados NÃO oferece dados em tempo real. Não coloque isso na apresentação. Não coloque NENHUM ponto negativo da Bet Dados.

Não tenho QR Code, então não peça para o usuário scannear um QR Code. Peça apenas para clicar no botão e assinar a Bet Dados. Explique que, após o pagamento,
o link de acesso ao grupo no Telegram será enviado por e-mail para o usuário.

COLOQUE IMAGEM APENAS NA CAPA E NA CONTRACAPA, NÃO COLOQUE MAIS NENHUMA IMAGEM NA APRESENTAÇÃO!

[ DADOS DA PARTIDA ]

Duelo: {clubes.confronto}

Competição: {clubes.campeonato}

Data: {clubes.data}

Hora: {clubes.hora}

Segue abaixo as estatísticas para a apresentação:

Temporada atual

{conv['tabela']}

Todas as informações abaixo são baseadas nos últimos 10 jogos do(a) {clubes.casa}. Deixe isso explícito e coloque um asterisco em todos os slides informando que os dados são das últimas 10 partidas do(a) {clubes.casa}.

Desempenho recente

{conv['tabela_recente']}

Primeiro tempo

{conv['tabela_tempo1']}

Segundo tempo

{conv['tabela_tempo2']}

Desempenho nos 90 minutos

{conv['tabela_completo']}

Clean sheets e clean goals
{conv['df_cleans']}

Desempenho em casa do(a) {clubes.casa}
{conv['desempenho_casa']}

Confronto direto desde 2020
{conv['desempenho_confronto']}

Confronto direto - últimos 10 jogos
{conv['historico_recente']}

Mais minutos

{conv['mais_minutos']}

Artilharia

{conv['gols']}

Assistências

{conv['assistencias']}

Participação em gols

{conv['participacao_gols']}

Rating total

{conv['rating']}

Rating médio

{conv['rating_medio']}

Grandes chances criadas

{conv['grandes_chances']}

Passes chave

{conv['passes_chave']}

xG total

{conv['xg']}

xG médio

{conv['xg_medio']}
"""

In [3435]:
import os

# === Descobre a pasta do projeto ===
try:
    # pasta onde este script .py está salvo
    pasta_projeto = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # fallback: pasta atual (para notebook/console)
    pasta_projeto = os.getcwd()

# === Monta o caminho do arquivo dentro da pasta do projeto ===
caminho_arquivo = os.path.join(pasta_projeto, "prompt.txt")

# === Salva como .txt (UTF-8) ===
with open(caminho_arquivo, "w", encoding="utf-8", newline="\n") as f:
    f.write(texto)

print(f"Arquivo .txt salvo com sucesso em: {caminho_arquivo}")


Arquivo .txt salvo com sucesso em: c:\Users\stefa\Desktop\bet_dados\prompt.txt


# Finalizando

In [3436]:
print("Finalizado")

Finalizado
